This file contains all the necessary functions to
1. Pull financial statements of publicly traded companies from [FMP](https://site.financialmodelingprep.com/)
2. Perform single DCF and Monte Carlo DCF valutaion  

### Using the fmpsdk library to pull finacials 
1. Library: https://github.com/daxm/fmpsdk
2. Data Provider: https://financialmodelingprep.com/

In [49]:
### you API KEY
my_fmp_api= "126a20a36ef3b5037c9dd7e329193476"
fred_key_api = '2f686365afa66dee66be93ae5cfd196b'

<IPython.core.display.Javascript object>

In [50]:
### import packages
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas_datareader
import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats
import sys
from fredapi import Fred
from numpy_ext import rolling_apply
import yfinance as yf
import fmpsdk
import openturns as ot

pd.options.display.max_columns=1000
pd.options.display.max_rows= 200
pd.options.display.float_format = '{:,.2f}'.format

<IPython.core.display.Javascript object>

In [51]:
def companies_profile_retriever_in_database():
    url_companies_profile_xlsx = "https://docs.google.com/spreadsheets/d/1FLGAWlL8edjUzZ2gC77PmCTaxLcgY0pjQwYmuT9dAik"
    df_ticker_profile = pd.read_excel(url_companies_profile_xlsx+'/export')
    return(df_ticker_profile)

<IPython.core.display.Javascript object>

In [52]:
# Adjust scroll-in-the-scroll in the entire Notebook 
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 850})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

<IPython.core.display.Javascript object>

In [53]:
def data_frame_flattener(df_data):
    df=df_data.copy()
    try:
        df.columns=[' '.join(map(str,col)).strip() for col in df.columns.values]
    except:
        pass
    return(df)

def column_suffix_adder(df_data,
                        list_of_columns_to_add_suffix_on,
                        suffix):
    """Add specific siffix to specific columns"""
    df=df_data.copy()
    ### Add suffix or prefix to certain columns rename all columns
    new_names = [(i,i+suffix) for i in df[list_of_columns_to_add_suffix_on].columns.values]
    df.rename(columns = dict(new_names), inplace=True)
    return(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
### The below API function is to retreive data from financialmodelingprep.com
#!/usr/bin/env python
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.
    Parameters
    ----------
    url : str
    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
### function to read googlesheet
def google_sheet_reader(url,
                        sheet_name):
  from gspread_dataframe import get_as_dataframe
  from google.colab import auth
  from google.auth import default
  import gspread
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  workbook = gc.open_by_url(url)
  sheet = workbook.worksheet(sheet_name)
  return(get_as_dataframe(sheet,parse_dates=True))


def google_sheet_savor(df,
                       url,
                       sheet_name):
  from gspread_dataframe import set_with_dataframe
  from google.colab import auth
  from google.auth import default
  import gspread
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)
  workbook = gc.open_by_url(url)
  ### Creat a workshet
  workbook.add_worksheet(sheet_name,1,1)
  ### Get the created worksheet
  created_worksheet = workbook.worksheet(sheet_name)
  set_with_dataframe(created_worksheet,df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
def companies_profile_retriever(list_of_symbol,
                                fmp_api):
    list_of_df_companies_profile=[]
    for symbol in list_of_symbol:
        try:
            df = pd.DataFrame(get_jsonparsed_data("https://financialmodelingprep.com/api/v3/profile/"+ symbol +"?apikey="+fmp_api))
            list_of_df_companies_profile.append(df)
        except:
            pass
    print(len(list_of_df_companies_profile)," Companies profile retrieved from FMP")
    if len(list_of_df_companies_profile)>0:
        df_ticker_profile = pd.concat(list_of_df_companies_profile,ignore_index=True)
        df_ticker_profile['currency']=df_ticker_profile['currency'].str.strip().str.upper()
        df_ticker_profile['sector']=df_ticker_profile['sector'].str.strip().str.upper()
        df_ticker_profile['industry']=df_ticker_profile['industry'].str.strip().str.upper()
        ### If the sector or  Industry is empty make it OTHER
        df_ticker_profile.loc[df_ticker_profile['industry'] == '','industry'] = 'OTHER'
        df_ticker_profile.loc[df_ticker_profile['sector'] == '','sector'] = 'OTHER'
        ### If the currency is missing make is USD and correct for bad currency spellings
        df_ticker_profile['currency'] = df_ticker_profile['currency'].replace({"ZAC":"ZAR",
                                                                               "ILA":"ILS",
                                                                               "":"USD"})
    else:
        print("Reading comapnies profile from the excel database")
        df_ticker_profile = companies_profile_retriever_in_database()
    return(df_ticker_profile)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
def data_retriver_from_fred(list_of_fatcors,
                            start_date = dt.datetime(2000,1,1),
                            fred_api="fred_api"):
    df_econ_data = pandas_datareader.data.DataReader(list_of_fatcors,
                                                     start=start_date,
                                                     data_source='fred')
    ## Faltten the data and drop null values
    df_econ_data = pd.melt(df_econ_data.reset_index(),
                           id_vars='DATE').dropna(subset=['value'])
    ### Get the attributes of each factors from Fred
    fred = Fred(api_key=fred_api)
    list_of_id_df=[]
    for id in list_of_fatcors:
        list_of_id_df.append(pd.DataFrame(fred.get_series_info(id)).T)
    df_econ_factor_attributes=pd.concat(list_of_id_df,ignore_index=True)
    df_econ_data = pd.merge(df_econ_data.rename(columns={'variable':'id'}),
                            df_econ_factor_attributes,
                            left_on='id',
                            right_on='id',
                            how='left')
    df_econ_data.rename(columns={"DATE":'Date'},inplace=True)
    return(df_econ_data)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
def historical_exchage_rate_provider_from_yahoo_finance(
    list_of_currencies= ['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """Get Historical Daily Exchange rate of currencies to USD"""
    ##import yfinance as yf
    ##import pandas as pd
    if len(list_of_currencies)==1:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')[['Adj Close']].rename(
            columns={"Adj Close":list_of_currencies[0]+"=X"})
    else:
        df_currency_ex_rate = yf.download([c+"=X" for c in list_of_currencies],
                                          period='max')['Adj Close']
    ### Melt the currency rates
    df_currency_ex_rate = pd.melt(df_currency_ex_rate.reset_index(),
        id_vars='Date',
        value_name='currency_rate',
        var_name='currency')
    ### Drop missing rates
    df_currency_ex_rate.dropna(subset=['currency_rate'],
                               inplace=True)
    ### trim the last 2 character of Currency Name '=X'
    df_currency_ex_rate["currency"]=df_currency_ex_rate["currency"].apply(lambda x: x[:-2])
    ### Fill in missing values of exchange rate wit previous available rate
    df_currency_ex_rate = df_currency_ex_rate.set_index("Date").groupby(
        ['currency']).resample('1D').ffill().drop(['currency'],axis=1).reset_index()

    df_currency_ex_rate = df_currency_ex_rate.sort_values(['currency','Date'])
    df_currency_ex_rate['source']='yahoo'
    return(df_currency_ex_rate)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
def historical_exchage_rate_provider(
    list_of_currencies=['USD','TWD','AUD','PLN',
                         'CZK','CNY','MXN','CHF',
                         'KRW','QAR','NZD','ILS',
                         'RUB','IDR','TRY','JPY',
                         'CAD','HKD','ZAR','MYR',
                         'BRL','SGD','INR','DKK',
                         'THB','SEK','NOK','GBP',
                         'EUR','CLP']):
    """The excel file named currency.xlsx has all the currency and their respective ticker symbol in FRED and Yahoo Finance.
    This functions take those tickers symbols and pull historical currency exchange rate from FED. Then pulls exchange rate from 
    Yahoo finance and return aggretated exchange rate from those 2 sources."""
    ##import datetime as dt
    ##import pandas as pd
    ##import pandas_datareader
    ##import yfinance as yf
    ### Get the possible currencies from your data base
    #df_currency = pd.read_excel(path_of_data+"Foreign Exchange Rates/currency.xlsx",sheet_name="currency")
    url_currency = "https://docs.google.com/spreadsheets/d/14t5Yilm6l34AFyb67k3v56R9OpDHZwk8"
    df_currency = pd.read_excel(url_currency+'/export',sheet_name="currency")
    ### Determine which currencies to pull from Fred
    set_of_ticker_currency_to_pull_from_fred = set(df_currency[(df_currency['currency'].isin(list_of_currencies)) & 
                                                               (df_currency['source']=='fred')]['ticker'])
    
    print("Retriving Exchange Rates from FRED")
    df_ex_Rate_fred= pandas_datareader.data.DataReader(set_of_ticker_currency_to_pull_from_fred,
                                                       data_source='fred',
                                                       start=dt.datetime(1950,1,1)).reset_index()
    print("Retriving Exchange Rates from FRED Complete",df_ex_Rate_fred.shape)
    ### Orgonize dataframe
    df_ex_Rate_fred = df_ex_Rate_fred.rename(columns={"DATE":"Date"}).set_index("Date")
    ### Melt the data
    df_ex_Rate_fred = pd.melt(df_ex_Rate_fred.reset_index(),
                                id_vars='Date',
                                value_name='currency_rate',
                                var_name='ticker')
    ### Drop missing rates
    df_ex_Rate_fred.dropna(subset=['currency_rate'],inplace=True)
    ### Resample to have a rate for every possible date in your date range for each currency
    df_ex_Rate_fred = df_ex_Rate_fred.set_index("Date").groupby(['ticker']).resample('1D').ffill().drop(['ticker'],
                                                                                           axis=1).reset_index()
    ### Get the currency standard abbreviation and as well as its target rate 
    df_ex_Rate_fred = pd.merge(df_ex_Rate_fred,
                           df_currency[["ticker","toUSD","currency"]],
                           left_on=['ticker'],
                           right_on=['ticker'],
                           how='left')
    ### Some of thses ticker you pulled from the Fred are from 1 Foreign currency to USD. 
    ### Most of them are from USD to Foreign Currecny. Make all of them from from USD to Foreign Currecny to keep it consistant with Yahoo Finance
    df_ex_Rate_fred.loc[df_ex_Rate_fred['toUSD']==True,'currency_rate'] = 1.0/df_ex_Rate_fred['currency_rate']
    df_ex_Rate_fred = df_ex_Rate_fred[['currency','Date','currency_rate']]
    df_ex_Rate_fred['source']='fred'
    
    ####Pull exchange rate from Yhaoo Finance
    print("Retriving Exchange Rates from YahooFinance")
    #print(list_of_currencies)
    df_ex_Rate_yahoo = historical_exchage_rate_provider_from_yahoo_finance(list_of_currencies)
    ### Creata a dataframe for USD to USD exchange rate
    df_ex_rate_usd = pd.DataFrame(pd.date_range(start=dt.datetime(1950,1,1),
                                                end=dt.datetime.now()+ dt.timedelta(days=5),
                                                freq='D'),columns=['Date'])
    df_ex_rate_usd['currency'] = 'USD'
    df_ex_rate_usd['currency_rate'] = 1.00
    
    ### Concatinate Data from FED and Yahoo Finance and USD Exchange Rate
    df_ex_rate = pd.concat([df_ex_Rate_fred,
                            df_ex_Rate_yahoo,
                            df_ex_rate_usd],
                           ignore_index=True).sort_values(['currency','Date','source'])
    df_ex_rate=df_ex_rate.drop_duplicates(subset=['currency','Date'],
                                          keep='last')
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate = df_ex_rate.set_index('Date').groupby(['currency']).resample('1D').ffill().drop(
        ['currency'],axis=1).reset_index()
    df_ex_rate.reset_index(drop=True,inplace=True)
    df_ex_rate.drop(['source'],axis=1,inplace = True)
    return(df_ex_rate)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
def tnd_calculator(df_data,
                    days='360D',
                    suffix='TTM',
                   growth='YoY',
                   func='sum',
                   col_to_compute_growth=[]):
    """ignoring future wanrnign for now. This mean and sum has to be revised """
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    df = df_data.copy()
    df['date_for_groupby'] = df['date'].copy()
    df.add_suffix("TTM").reset_index()
    df_agg= df.groupby(['date_for_groupby']).apply(lambda x: pd.concat([df])).drop(
        ['date_for_groupby'],axis=1).reset_index().drop(['level_1'],axis=1)
    df_agg = df_agg[df_agg['date'] <= df_agg['date_for_groupby']]
    if func == 'sum':
        df_agg = df_agg.set_index("date").groupby(['symbol','date_for_groupby']).resample(days,closed='right',origin='end').sum()
    if func == 'mean':
        df_agg = df_agg.set_index("date").groupby(['symbol','date_for_groupby']).resample(days,closed='right',origin='end').mean()
    df_agg = df_agg.add_suffix(suffix).reset_index()
    # df_agg['dateDiff']= df_agg.groupby(['symbol','date_for_groupby'])['date'].diff().dt.days
    if len(col_to_compute_growth)>1:
        for col in col_to_compute_growth:
            df_agg[col+growth] = df_agg[col+suffix].pct_change()
    df_agg = df_agg[df_agg['date_for_groupby'] == df_agg['date']].reset_index(drop=True) 
    return(df_agg)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
def dna_caclulator(spend_date,
                   amortization_start_date,
                   amortization_end_date,
                   principal):
    """amortize R&D on linear basis"""
    date_range_before_amortization_begin = pd.date_range(start=spend_date,
                                                         end = amortization_start_date,
                                                         freq='1D',
                                                         inclusive='left')
    df_befor_amortization = pd.DataFrame({'date':date_range_before_amortization_begin})
    df_befor_amortization['principal'] = 0
    df_befor_amortization.loc[df_befor_amortization['date'] == spend_date,'principal'] = principal
    df_befor_amortization['amortization'] = 0.0
    df_befor_amortization['unamortized'] = principal
    df_befor_amortization['cumAmortization'] = 0.0
    date_range = pd.date_range(start=amortization_start_date,
                                   end = amortization_end_date,
                                   freq='1D')
    value_of_amortization_each_day = principal/len(date_range)
    df = pd.DataFrame({'date':date_range})
    df['principal'] = 0
    df['amortization'] = value_of_amortization_each_day
    df['cumAmortization'] = df['amortization'].cumsum()
    df['unamortized'] = principal- df['cumAmortization']
    df_res = pd.concat([df_befor_amortization,df],ignore_index=True)
    return(df_res)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### financial_statments_preparator

In [62]:
def financial_statments_preparator(company_symbol,
                                   fmp_api,
                                   period="quarter",
                                   limit=1000,
                                   rnd_amortization_period_begin=0,
                                   rnd_amortization_period=3,
                                   convert_to_USD=True,
                                   df_currency_ex_rate=pd.DataFrame()):
    """Get All financial Statemnts (Income,Balancesheet,Cashflow) as well as trailing 12 month. All in One dataFrame.
    If convert_to_USD=True, you need to supply the exchange rate. This functions has to be revised.
    Currenly Amotrize R&D for 3 years. Should add parameter to it"""
    ##import pandas as pd
    ###import numpy as np
    ##import datetime as dt
    ##import fmpsdk
    df_currency_exchange_rate = df_currency_ex_rate.copy()
    df_income_statement = pd.DataFrame (fmpsdk.income_statement(apikey=fmp_api,
                                                  symbol=company_symbol,
                                                  period=period,
                                                  limit=1000))

    df_income_statement.drop(['cik','calendarYear'],axis=1,inplace=True)


    df_balance_sheet = pd.DataFrame(fmpsdk.balance_sheet_statement(apikey=fmp_api,
                                                      symbol=company_symbol,
                                                      period=period,
                                                      limit=1000))
    df_balance_sheet.drop(['cik','calendarYear'],axis=1,inplace=True)

    df_cash_flow = pd.DataFrame (fmpsdk.cash_flow_statement(apikey=fmp_api,
                                                            symbol=company_symbol,
                                                            period=period,
                                                            limit=1000))

    df_cash_flow.drop(['cik','calendarYear'],axis=1,inplace=True)

    ### Convert Date column data type to datatime
    df_income_statement["date"]=pd.to_datetime(df_income_statement["date"])
    df_balance_sheet["date"]=pd.to_datetime(df_balance_sheet["date"])
    df_cash_flow["date"]=pd.to_datetime(df_cash_flow["date"])

    ### Convert Date column data type to datatime
    df_income_statement["fillingDate"]=pd.to_datetime(df_income_statement["fillingDate"])
    df_balance_sheet["fillingDate"]=pd.to_datetime(df_balance_sheet["fillingDate"])
    df_cash_flow["fillingDate"]=pd.to_datetime(df_cash_flow["fillingDate"])
    ### There may be some instances where th filing date is null. Fillna with financial date but add 20 days.
    df_income_statement["fillingDate"]= df_income_statement["fillingDate"].fillna(df_income_statement['date']+ dt.timedelta(days=20))
    df_balance_sheet["fillingDate"]=df_balance_sheet["fillingDate"].fillna(df_balance_sheet['date']+ dt.timedelta(days=20))
    df_cash_flow["fillingDate"]=df_cash_flow["fillingDate"].fillna(df_cash_flow['date']+ dt.timedelta(days=20))

    ######################## Amortize R&D Expenses ######################################################
    ### determine the date which amortization begin and end
    df_income_statement['rndDateAmortizationBegin'] = df_income_statement['date'] +  pd.to_timedelta(365*rnd_amortization_period_begin, unit='d')
    df_income_statement['rndDateAmortizationEnd'] = df_income_statement['rndDateAmortizationBegin'] +  pd.to_timedelta(365*rnd_amortization_period, unit='d')

    ### Drive R&D Amortizationn table for each income statemnet via the dna_calculator
    df_income_statement['rndAmortizaionTable'] = df_income_statement.apply(lambda row:
                            dna_caclulator(spend_date=row['date'],
                                            amortization_start_date=row['rndDateAmortizationBegin'],
                                            amortization_end_date=row['rndDateAmortizationEnd'],
                                            principal=row['researchAndDevelopmentExpenses']
                                            ),axis=1)
    ### for every ticker gather the amortization tables
    df_dna = df_income_statement.groupby(['symbol'])['rndAmortizaionTable'].apply(lambda x: pd.concat(x.tolist())).reset_index()
    df_dna = df_dna.groupby(['symbol','date'])[['principal', 'amortization', 'unamortized']].sum().reset_index()
    set_of_symbol_date_zipped = set(
        zip(df_income_statement['symbol'],
            df_income_statement['date'])
        )
    df_dna.loc[df_dna[['symbol','date']].apply(tuple,axis=1).isin(set_of_symbol_date_zipped),
            'date_in_income_statement']  = df_dna['date']


    df_dna['date_in_income_statement'] = df_dna['date_in_income_statement'].bfill()
    df_dna['amortization_for_income_statemnet'] = df_dna.groupby(['date_in_income_statement'])['amortization'].cumsum()


    df_income_statement = pd.merge(df_income_statement,
                                df_dna[['date','symbol','unamortized','amortization_for_income_statemnet']].rename(columns={"unamortized":'rndUnamortized',
                                                                                                                            "amortization_for_income_statemnet":'rndAmortization'}),
                                left_on=['date','symbol'],
                                right_on=['date','symbol'],
                                how='left')
    

    df_income_statement['operatingIncomeAdjusted'] = df_income_statement['operatingIncome']  + df_income_statement['researchAndDevelopmentExpenses'] - df_income_statement['rndAmortization']
    df_income_statement['operatingExpensesAdjusted'] = df_income_statement['operatingExpenses'] - df_income_statement['researchAndDevelopmentExpenses'] + df_income_statement['rndAmortization']
    ############################################################## End of R&D Amortization
    
    #### Compute Metiercs Before Conversion 
    ## Adjusted OperatingIncome: Adjuste for R&D
    df_income_statement['ebit'] = df_income_statement['ebitda'] - df_income_statement['depreciationAndAmortization']
    df_income_statement['ebitAdjusted'] = df_income_statement['ebit']  + df_income_statement['researchAndDevelopmentExpenses'] - df_income_statement['rndAmortization']


    #### First Convert each financial statement to USD then compute metircs, like growth etc.
    if convert_to_USD:
        df_income_statement['reportedCurrency'] = df_income_statement['reportedCurrency'].fillna("USD")
        df_balance_sheet['reportedCurrency'] = df_balance_sheet['reportedCurrency'].fillna("USD")
        df_cash_flow['reportedCurrency'] = df_cash_flow['reportedCurrency'].fillna("USD")
        ### Compute 90 Days moving average
        df_currency_exchange_rate["currency_rateMA90"] = df_currency_exchange_rate.groupby(
            ['currency'])['currency_rate'].rolling(90).mean().reset_index(0,drop=True)
        df_currency_exchange_rate["currency_rateMA90"]= df_currency_exchange_rate["currency_rateMA90"].fillna(
            df_currency_exchange_rate["currency_rate"])
        ### Mereg ExRates with income statemnet
        df_income_statement = pd.merge(df_income_statement,
                                 df_currency_exchange_rate.rename(columns={"Date":"date",
                                                                           "currency":"reportedCurrency"}),
                                 left_on=['reportedCurrency','date'],
                                 right_on=['reportedCurrency','date'],
                                 how='left')
        df_income_statement['currency_rate']= df_income_statement['currency_rate'].fillna(1.0)
        df_income_statement['currency_rateMA90']= df_income_statement['currency_rateMA90'].fillna(1.0)
        
        ### Mereg ExRates with balacnesheet
        df_balance_sheet = pd.merge(df_balance_sheet,
                                 df_currency_exchange_rate.rename(columns={"Date":"date",
                                                                           "currency":"reportedCurrency"}),
                                 left_on=['reportedCurrency','date'],
                                 right_on=['reportedCurrency','date'],
                                 how='left')
        df_balance_sheet['currency_rate']=df_balance_sheet['currency_rate'].fillna(1.0)
        df_balance_sheet['currency_rateMA90']=df_balance_sheet['currency_rateMA90'].fillna(1.0)
        
        ### Mereg ExRates with cashflow
        df_cash_flow = pd.merge(df_cash_flow,
                                 df_currency_exchange_rate.rename(columns={"Date":"date",
                                                                           "currency":"reportedCurrency"}),
                                 left_on=['reportedCurrency','date'],
                                 right_on=['reportedCurrency','date'],
                                 how='left')
        df_cash_flow['currency_rate']=df_cash_flow['currency_rate'].fillna(1.0)
        df_cash_flow['currency_rateMA90']=df_cash_flow['currency_rateMA90'].fillna(1.0)
        set_of_columns_to_not_convert_to_usd = {'acceptedDate','date','fillingDate','finalLink','link','link_ISfillingDate',
                                                'grossProfitRatio','ebitdaratio', 'operatingIncomeRatio', 'incomeBeforeTaxRatio' ,'netIncomeRatio',
                                                'period','reportedCurrency','symbol','weightedAverageShsOut','weightedAverageShsOutDil',
                                                'currency_rate','currency_rateMA90','rndAmortizaionTable','rndDateAmortizationBegin', 'rndDateAmortizationEnd'}
        print("Converting",company_symbol,"to USD")
        ### Convert income statemnet to US
        for df_financial_in_usd in [df_income_statement,df_cash_flow,df_balance_sheet]:
            list_of_columns_to_convert_to_usd = df_financial_in_usd.drop(
            set(df_financial_in_usd.columns).intersection(set_of_columns_to_not_convert_to_usd),axis=1).columns.tolist()
            #print(list_of_columns_to_convert_to_usd)
            df_financial_in_usd[list_of_columns_to_convert_to_usd] =  df_financial_in_usd[
                                                                                          list_of_columns_to_convert_to_usd].divide(df_financial_in_usd['currency_rateMA90'],
                                                                                                                                    axis='index')

    # ############################## Balncesheet###########################################################
    ### Sort Balancesheet and compuet change in non cash working capital
    df_balance_sheet.sort_values("date",inplace=True)
    df_balance_sheet.reset_index(drop=True,inplace=True)
    df_balance_sheet['workingCapital']=df_balance_sheet['totalCurrentAssets'] - df_balance_sheet['totalCurrentLiabilities']
    df_balance_sheet['nonCashWorkingCapital']= ((df_balance_sheet['totalCurrentAssets'] - df_balance_sheet['cashAndShortTermInvestments']) -
                                                (df_balance_sheet['totalCurrentLiabilities'] - df_balance_sheet['shortTermDebt']))
    df_balance_sheet["changeInNonCashWorkingCapital"] = df_balance_sheet['nonCashWorkingCapital'].diff()
    df_balance_sheet['changeInDebt']= df_balance_sheet['totalDebt'].diff()
    
    ##########################Cashflow#######################################################################
    #compute CapEx
    df_cash_flow['capEx']= (df_cash_flow['capitalExpenditure'] + 
                            df_cash_flow['acquisitionsNet'])
    
    #Compute NextCapEx
    ### CapEx is coming from cashflow statemnt and it's sign in inverse (- money spent + moeny gained by selling assets etc.)
    df_cash_flow['netCapEx'] = df_cash_flow['capEx'] + df_cash_flow['depreciationAndAmortization']
    


    ############################# TTM ########################
    #### Get the TTM of Income Statement
    list_of_non_currency_cols_to_exclude_from_income_ttm= ['grossProfitRatio', 'ebitdaratio', 'operatingIncomeRatio',
                                                          'incomeBeforeTaxRatio', 'netIncomeRatio', 'weightedAverageShsOut',
                                                          'weightedAverageShsOutDil']
    if convert_to_USD:
        for el in ['currency_rate','currency_rateMA90']:
            list_of_non_currency_cols_to_exclude_from_income_ttm.append(el)
    df_income_statement_rolling_ttm = tnd_calculator(df_income_statement.drop(
        list_of_non_currency_cols_to_exclude_from_income_ttm,
        axis=1),
        col_to_compute_growth=['revenue','costOfRevenue','grossProfit',
                               'operatingIncome','operatingIncomeAdjusted','ebit','ebitda','sellingGeneralAndAdministrativeExpenses'])
    df_income_statement = pd.merge(df_income_statement,
                                   df_income_statement_rolling_ttm,
                                   left_on=["symbol","date"],
                                   right_on=["symbol","date"],
                                  how='left')
    
    df_income_statement['operatingIncomeRatio']=df_income_statement['operatingIncome']/df_income_statement['revenue']
    df_income_statement['operatingIncomeAdjustedRatio']=df_income_statement['operatingIncomeAdjusted']/df_income_statement['revenue']
    df_income_statement['ebitRatio']= df_income_statement['ebit']/df_income_statement['revenue']
    df_income_statement['ebitRatioAdjusted']= df_income_statement['ebitAdjusted']/df_income_statement['revenue']
    # df_income_statement['ebitdaRatio']= df_income_statement['ebitda']/df_income_statement['revenue']
    df_income_statement["grossProfitRatio"]=df_income_statement["grossProfit"]/df_income_statement["revenue"]
    df_income_statement["netIncomeRatio"]=df_income_statement["netIncome"]/df_income_statement["revenue"]
    #### Cost from Revenue
    df_income_statement['costOfRevenueRatio'] = df_income_statement['costOfRevenue']/df_income_statement['revenue']
    df_income_statement['sellingGeneralAndAdministrativeExpensesRatio'] = df_income_statement['sellingGeneralAndAdministrativeExpenses']/df_income_statement['revenue']
    df_income_statement['operatingExpensesRatio'] = df_income_statement['operatingExpenses']/df_income_statement['revenue']
    df_income_statement['operatingExpensesAdjustedRatio'] = df_income_statement['operatingExpensesAdjusted']/df_income_statement['revenue']
    df_income_statement['InterestExpenseRatio'] = df_income_statement['interestExpense']/df_income_statement['revenue']
    df_income_statement['researchAndDevelopmentExpensesRatio'] = df_income_statement['researchAndDevelopmentExpenses']/df_income_statement['revenue']
    df_income_statement['otherExpensesRatio'] = df_income_statement['otherExpenses']/df_income_statement['revenue']
    df_income_statement['rndAmortizationRatio'] = df_income_statement['rndAmortization']/df_income_statement['revenue']
    #df_income_statement['depreciationAndAmortizationRatio'] = df_income_statement['depreciationAndAmortization']/df_income_statement['revenue']
    
    ##TTTM
    df_income_statement['operatingIncomeRatioTTM']=df_income_statement['operatingIncomeTTM']/df_income_statement['revenueTTM']
    df_income_statement['ebitRatioTTM']= df_income_statement['ebitTTM']/df_income_statement['revenueTTM']
    df_income_statement['operatingIncomeAdjustedRatioTTM']=df_income_statement['operatingIncomeAdjustedTTM']/df_income_statement['revenueTTM']
    df_income_statement['ebitAdjustedRatioTTM']= df_income_statement['ebitAdjustedTTM']/df_income_statement['revenueTTM']
    df_income_statement['ebitdaRatioTTM']= df_income_statement['ebitdaTTM']/df_income_statement['revenueTTM']
    df_income_statement['costOfRevenueRatioTTM'] = df_income_statement['costOfRevenueTTM']/df_income_statement['revenueTTM']
    df_income_statement['operatingExpensesRatioTTM'] = df_income_statement['operatingExpenses']/df_income_statement['revenueTTM']
    df_income_statement["grossProfitRatioTTM"]=df_income_statement["grossProfitTTM"]/df_income_statement["revenueTTM"]
    df_income_statement["netIncomeRatioTTM"]=df_income_statement["netIncomeTTM"]/df_income_statement["revenueTTM"]

    
    ### Get the TTM of Cash Flow
    if convert_to_USD:
        df_cash_flow_rolling_ttm = tnd_calculator(df_cash_flow.drop(['currency_rate','currency_rateMA90'],axis=1))
    else:
        df_cash_flow_rolling_ttm = tnd_calculator(df_cash_flow)
    df_cash_flow = pd.merge(df_cash_flow,
                            df_cash_flow_rolling_ttm,
                            left_on=["symbol","date"],
                            right_on=["symbol","date"],
                            how='left')
    
    ### Compute change in non cash working capital and Change in Debt for the last trailing 12 months
    ### Get the TTM of Balancesheet
    df_balance_sheet_rolling_ttm = tnd_calculator(df_balance_sheet[['date','fillingDate','symbol',
                                                                    'changeInNonCashWorkingCapital',
                                                                    'changeInDebt']])
    df_balance_sheet = pd.merge(df_balance_sheet,
                                df_balance_sheet_rolling_ttm,
                                left_on=["symbol","date"],
                                right_on=["symbol","date"],
                                how='left')
    
    set_of_dates_with_no_full_match_income_vs_cf = set(df_income_statement['date']) - set(df_cash_flow['date'])
    set_of_dates_with_no_full_match_income_vs_bs = set(df_income_statement['date']) - set(df_balance_sheet['date'])
    ### Print the following for trouble shoting
    # if len(set_of_dates_with_no_full_match_income_vs_cf)>0:
    #     print("Dates with no full mactch on financial statments income vs cf", set_of_dates_with_no_full_match_income_vs_cf ,list(set(df_income_statement['symbol'])))
    # if len(set_of_dates_with_no_full_match_income_vs_bs)>0:
    #     print("Dates with no full mactch on financial statments income vs bs", set_of_dates_with_no_full_match_income_vs_bs ,list(set(df_income_statement['symbol'])))

    ### Combine all statements into one dataframe name it df_financials
    df_financials = pd.merge(df_income_statement,
                             df_balance_sheet,
                             left_on=['symbol','date'],
                             right_on=['symbol','date'],
                            how="outer",
                            suffixes=("_IS","_BS")).sort_values(["date","symbol"])
    ### If income statment date is null after merging with BS, fillna with with filling date of BS
    df_financials['fillingDate_BS'] = df_financials['fillingDate_BS'].fillna(df_financials['fillingDate_IS'])
    df_financials['fillingDate_IS'] = df_financials['fillingDate_IS'].fillna(df_financials['fillingDate_BS'])
    
    df_financials = pd.merge(df_financials,
                             df_cash_flow,
                             left_on=['symbol','date'],
                             right_on=['symbol','date'],
                            how="outer",
                            suffixes=("","_CF")).sort_values(["date","symbol"],
                                                             ascending=True)
    
    ### If income statment date or BS is null after merging with CF, fillna with with filling date of CF
    df_financials['fillingDate_BS'] = df_financials['fillingDate_BS'].fillna(df_financials['fillingDate'])
    df_financials['fillingDate_IS'] = df_financials['fillingDate_IS'].fillna(df_financials['fillingDate'])
    df_financials['fillingDate'] = df_financials['fillingDate'].fillna(df_financials['fillingDate_IS'])
    ### There may be some time where the balancesheet/CF or Income statmnet have slightly different date. Drop those that are
    ### less than 10 days apart.
    df_financials.ffill(inplace=True)
    df_financials['date_diff'] = (df_financials['date'].diff(1)).dt.days
    df_financials['date_diff_1']= df_financials['date_diff'].shift(-1).fillna(10000)
    rows_to_drop = df_financials[df_financials['date_diff_1']<25].index
    df_financials.drop(rows_to_drop,axis=0,inplace=True)
    df_financials.reset_index(drop=True,inplace=True)
    ### In case if the filling date is duplicated, take the date and add 20 days to it
    df_financials.loc[df_financials['fillingDate_IS'].duplicated(keep=False),
                      "fillingDate_IS"] = df_financials['date'] + dt.timedelta(days=20)
    df_financials.loc[df_financials['fillingDate_BS'].duplicated(keep=False),
                      "fillingDate_BS"] = df_financials['date'] + dt.timedelta(days=20)
    df_financials.loc[df_financials['fillingDate'].duplicated(keep=False),
                      "fillingDate"] = df_financials['date'] + dt.timedelta(days=20)
    ### Create one Col as Financial Currency
    df_financials['FinancialCurrency'] = df_financials['reportedCurrency_IS'].fillna(df_financials['reportedCurrency_BS'])
    df_financials['FinancialCurrency'] = df_financials['FinancialCurrency'].fillna(df_financials['reportedCurrency'])
  
    # #### Compute Metrics
    df_financials['diluatedShares'] = df_financials['weightedAverageShsOutDil'] - df_financials['weightedAverageShsOut']
    df_financials["effectiveTaxRate"]=df_financials["incomeTaxExpense"]/df_financials["incomeBeforeTax"]
    df_financials["effectiveTaxRateTTM"]=df_financials["incomeTaxExpenseTTM"]/df_financials["incomeBeforeTaxTTM"]
    ### Tax rate could jump aournd, so use the median over the last 16 quartertes
    df_financials["effectiveTaxRateTTM"] = df_financials["effectiveTaxRateTTM"].fillna(df_financials["effectiveTaxRateTTM"])
    df_financials["medianEffectiveTaxRateTTM"] = df_financials["effectiveTaxRateTTM"].rolling(16).median()
    df_financials["medianEffectiveTaxRateTTM"] = df_financials["effectiveTaxRateTTM"].fillna(df_financials["effectiveTaxRateTTM"])
    
    
    # df_financials.loc[df_financials["operatingIncome"]>0,
    #                   'operatingIncomeAfterTax'] = df_financials["operatingIncome"]*(1-df_financials["medianEffectiveTaxRateTTM"])
    # df_financials.loc[df_financials["operatingIncome"]<0,
    #               'operatingIncomeAfterTax'] = df_financials["operatingIncome"]
    
    # df_financials.loc[df_financials["operatingIncomeTTM"]>0,
    #               'operatingIncomeTTMAfterTax'] = df_financials["operatingIncomeTTM"]*(1-df_financials["medianEffectiveTaxRateTTM"]) 
    # df_financials.loc[df_financials["operatingIncomeTTM"]<0,
    #               'operatingIncomeTTMAfterTax'] = df_financials["operatingIncomeTTM"]

    for income in ['operatingIncome','operatingIncomeTTM','operatingIncomeAdjusted','operatingIncomeAdjustedTTM']:
            df_financials.loc[df_financials[income]>0,
                              income+'AfterTax'] = df_financials[income]*(1-df_financials["medianEffectiveTaxRateTTM"])
            df_financials.loc[df_financials[income]<0,
                              income+'AfterTax'] = df_financials[income]

    ### Estimate ADjusted CapEx
    ### R&D come fron income statemnet as expense with + sign
    df_financials['capExAdjusted'] = (df_financials['capitalExpenditure'] + 
                                  df_financials['acquisitionsNet'] - 
                                  df_financials['researchAndDevelopmentExpenses'])
    
    df_financials['capExAdjustedTTM'] = (df_financials['capitalExpenditureTTM'] + 
                                         df_financials['acquisitionsNetTTM'] - 
                                         df_financials['researchAndDevelopmentExpensesTTM'])
    
    df_financials['netCapExAdjusted'] = df_financials['capExAdjusted'] + df_financials['depreciationAndAmortization'] + df_financials['rndAmortization']
    df_financials['netCapExAdjustedTTM'] = df_financials['capExAdjustedTTM'] + df_financials['depreciationAndAmortizationTTM'] + df_financials['rndAmortizationTTM']

    ### Estimte Firm Reinvestment
    df_financials['frimReinvestment'] = (df_financials['netCapEx'] - df_financials['changeInNonCashWorkingCapital'])
    df_financials['frimReinvestmentAdjusted'] = (df_financials['netCapExAdjusted']- df_financials['changeInNonCashWorkingCapital'])

    df_financials['frimReinvestmentTTM'] = (df_financials['netCapExTTM']- df_financials['changeInNonCashWorkingCapitalTTM'])
    df_financials['frimReinvestmentAdjustedTTM'] = (df_financials['netCapExAdjustedTTM']- df_financials['changeInNonCashWorkingCapitalTTM'])

    df_financials['frimReinvestmentRate']= df_financials['frimReinvestment']/df_financials['operatingIncomeAfterTax']
    df_financials['frimReinvestmentRateAdjusted']= df_financials['frimReinvestmentAdjusted']/df_financials['operatingIncomeAdjustedAfterTax']

    df_financials['frimReinvestmentRateTTM'] = df_financials['frimReinvestmentTTM']/df_financials['operatingIncomeTTMAfterTax']
    df_financials['frimReinvestmentRateAdjustedTTM'] = df_financials['frimReinvestmentAdjustedTTM']/df_financials['operatingIncomeAdjustedTTMAfterTax']

    ### Estimate Equity Reinvesment
    df_financials['equityReinvestment'] = (df_financials['netCapEx'] -
                                           df_financials['changeInNonCashWorkingCapital']+
                                           df_financials['changeInDebt'])
    df_financials['equityReinvestmentTTM'] = (df_financials['netCapExTTM'] -
                                              df_financials['changeInNonCashWorkingCapitalTTM']+
                                              df_financials['changeInDebtTTM'])
    ### Estimate Equity Reinvesment Rate
    df_financials['equityReinvestmentRate']=df_financials['equityReinvestment']/df_financials['netIncome']
    df_financials['equityReinvestmentRateTTM']=df_financials['equityReinvestmentTTM']/df_financials['netIncomeTTM']
    
    ### Estimate Free Cash Flow to the Frim traditional way:
    ### FCFF = EBIT(1-Effective Tax Rate) + Depreciatio & Amortization - Cap Ex - Acquisition
    df_financials['FCFF'] =  (df_financials['operatingIncomeAfterTax']+
                              df_financials['netCapEx']-
                              df_financials['changeInNonCashWorkingCapital'])
    df_financials['FCFFTTM'] =  (df_financials['operatingIncomeTTMAfterTax']+
                                 df_financials['netCapExTTM']-
                                 df_financials['changeInNonCashWorkingCapitalTTM'])
    ### Estimate Cash Flow to equity: FCFE
    ## Free cash flow to equity = Net income − (Capital expenditures − Depreciation) − (Change in noncash working capital) + (New debt issued − Debt repayments)
    df_financials['FCFEPreDebt'] = (df_financials['netIncome'] + 
                                    df_financials['netCapEx'] -
                                    df_financials['changeInNonCashWorkingCapital'])
    
    df_financials['FCFEPreDebtTTM'] = (df_financials['netIncomeTTM'] + 
                                       df_financials['netCapExTTM'] -
                                       df_financials['changeInNonCashWorkingCapitalTTM'])

    df_financials['FCFEAfterDebt'] = (df_financials['netIncome'] + 
                                      df_financials['netCapEx'] -
                                      df_financials['changeInNonCashWorkingCapital']+
                                      df_financials['changeInDebt'])
    
    df_financials['FCFEAfterDebtTTM'] = (df_financials['netIncomeTTM'] + 
                                         df_financials['netCapExTTM'] -
                                         df_financials['changeInNonCashWorkingCapitalTTM']+
                                         df_financials['changeInDebtTTM'])
    
    # df_financials["grossProfitRatioTTM"]=df_financials["grossProfitTTM"]/df_financials["revenueTTM"]
    # df_financials["operatingIncomeRatioTTM"]=df_financials["operatingIncomeTTM"]/df_financials["revenueTTM"]
    # df_financials["netIncomeRatioTTM"]=df_financials["netIncomeTTM"]/df_financials["revenueTTM"]
    #### Estimate cost of doing business as function of revenue
    # df_financials['operatingExpensesRatio'] = df_financials['operatingExpenses']/df_financials['revenue']
    # df_financials['ebit'] = df_financials['ebitda'] - df_financials['depreciationAndAmortization']
    # df_financials['ebitRatio']=df_financials['ebit']/df_financials['revenue']
    # df_financials['costOfRevenueRatio'] = df_financials['costOfRevenue']/df_financials['revenue']
    # df_financials['operatingExpensesRatio'] = df_financials['operatingExpenses']/df_financials['revenue']
    # df_financials['operatingIncomeCal'] = df_financials['revenue'] - df_financials['costOfRevenue'] - df_financials['operatingExpenses']
    # df_financials['operatingIncomeCalRatio']=df_financials['operatingIncomeCal']/df_financials['revenue']
    
    # ### PCT Change
    df_financials['revenueChange'] = df_financials['revenue'].pct_change()
    df_financials['revenueTTMChange']=df_financials['revenueTTM'].pct_change()
    df_financials['costOfRevenueTTMChange']=df_financials['costOfRevenueTTM'].pct_change()
    df_financials['grossProfitTTMChange']=df_financials['grossProfitTTM'].pct_change()
    df_financials['grossProfitChange']=df_financials['grossProfit'].pct_change()
    df_financials['operatingIncomeTTMChange']=df_financials['operatingIncomeTTM'].pct_change()
    df_financials['operatingIncomeChange']=df_financials['operatingIncome'].pct_change()
    df_financials['operatingIncomeAdjustedTTMChange'] = df_financials['operatingIncomeAdjustedTTM'].pct_change()
    df_financials['operatingIncomeAdjustedChange']=df_financials['operatingIncomeAdjusted'].pct_change()
    df_financials['netIncomeTTMChange']=df_financials['netIncomeTTM'].pct_change()
    df_financials['netIncomeChange']=df_financials['netIncome'].pct_change()

    ### Operating leverage refers to the proportion of the total costs of the firm that are fixed. Other things remaining equal, higher operating
    ### leverage results in greater earnings variability which in turn results in higher betas.
    df_financials['sellingGeneralAndAdministrativeExpensesChange'] = df_financials['sellingGeneralAndAdministrativeExpenses'].pct_change()
    df_financials['variabilitySG&A'] = df_financials['sellingGeneralAndAdministrativeExpensesChange']/df_financials['revenueChange']
    df_financials['variabilitySG&AYoY'] = df_financials['sellingGeneralAndAdministrativeExpensesYoY']/df_financials['revenueYoY']

    df_financials['operatingLeverage'] = df_financials['operatingIncomeChange']/df_financials['revenueChange']
    df_financials['operatingLeverageYoY'] = df_financials['operatingIncomeYoY']/df_financials['revenueYoY']
    df_financials['operatingLeverageAdjusted'] = df_financials['operatingIncomeAdjustedChange']/df_financials['revenueChange']
    df_financials['operatingLeverageAdjustedYoY'] = df_financials['operatingIncomeAdjustedYoY']/df_financials['revenueYoY']

    df_financials["interestExpenseCoverageRatio"] = df_financials['operatingIncomeTTM']/df_financials['interestExpenseTTM']
    df_financials["debtToOperatingIncome"] = df_financials['totalDebt']/df_financials['operatingIncomeTTM']
    df_financials["debtToOperatingIncomeAdjusted"] = df_financials['totalDebt']/df_financials['operatingIncomeAdjustedTTM']

    df_financials["currentRatio"] = df_financials['totalCurrentAssets']/df_financials['totalCurrentLiabilities']
    df_financials["longTermRatio"] = (df_financials['totalAssets']-df_financials['goodwill'])/df_financials['totalLiabilities']
    df_financials["totalEquityExcGoodwill"] = (df_financials['totalAssets']-df_financials['totalLiabilities']-df_financials['goodwill'])
    df_financials["debtToAsset"] = df_financials['totalDebt']/(df_financials['totalAssets']-df_financials['goodwill'])
    
    ### I reduce long term investment values by 15% due to capital gain tax, liquidiation cost etc.
    df_financials["totalCash"]=(df_financials["cashAndCashEquivalents"] + 
                                df_financials["shortTermInvestments"] +
                                (df_financials["longTermInvestments"]*.85))
    
    df_financials["investedCapitalBookValue"] = (df_financials["totalLiabilitiesAndStockholdersEquity"] -
                                                         df_financials["totalCash"])

    df_financials["investedCapitalBookValueAdjusted"] = (df_financials["totalLiabilitiesAndStockholdersEquity"] + 
                                                         df_financials['rndUnamortized'] -
                                                         df_financials["totalCash"])
    
    df_financials['totalLiabilitiesAndStockholdersEquityAdjusted'] = (df_financials["totalLiabilitiesAndStockholdersEquity"]+ df_financials["rndUnamortized"])
    
    df_financials['investedCapitalBookValueDebtEquity']=(df_financials['totalStockholdersEquity']+
                                                         df_financials['totalDebt']-
                                                         df_financials['cashAndShortTermInvestments'])
    
    df_financials['investedCapitalBookValueDebtEquityAdjusted']=(df_financials['totalStockholdersEquity']+
                                                                 df_financials['rndUnamortized']+
                                                                 df_financials['totalDebt']-
                                                                 df_financials['cashAndShortTermInvestments'])

    ### Compute averega asset and liabilites through teh year. Then to compuote ROIC, ROE.
    list_of_invested_capital_column= ['totalLiabilitiesAndStockholdersEquity',
                                      'totalLiabilitiesAndStockholdersEquityAdjusted',
                                      'investedCapitalBookValue',
                                      'investedCapitalBookValueAdjusted',
                                      'investedCapitalBookValueDebtEquity',
                                      'investedCapitalBookValueDebtEquityAdjusted',
                                      'totalStockholdersEquity',
                                      'cashAndShortTermInvestments',
                                      'totalCash',
                                      'totalDebt']
    
    df_ave_asset_and_liabilities = tnd_calculator(df_financials[['date','symbol']+list_of_invested_capital_column],
                                                  days='360D',
                                                  suffix='TTMAve',
                                                  growth='YoY',
                                                  func='mean',
                                                  col_to_compute_growth=[])

    df_financials = pd.merge(df_financials,
                             df_ave_asset_and_liabilities,
                             left_on=["symbol","date"],
                             right_on=["symbol","date"],
                             how='left',
                             suffixes=("","_capital")).copy()

    df_financials["costOfDebtBackwardLooking"] = (df_financials['interestExpense']/df_financials['totalDebt'])*4
    ### Traditional Metrics
    df_financials["salesToCapitalRatioQ"] = df_financials['revenue']/df_financials['investedCapitalBookValueDebtEquity']
    df_financials["salesToCapitalRatio"] = df_financials['revenueTTM']/df_financials['investedCapitalBookValueDebtEquityTTMAve']
    df_financials["salesToCapitalAdjustedRatio"] = df_financials['revenueTTM']/df_financials['investedCapitalBookValueDebtEquityAdjustedTTMAve']

    df_financials['salesToAssetRatio'] = df_financials['revenueTTM']/df_financials['totalLiabilitiesAndStockholdersEquityTTMAve']
    df_financials['salesToAssetAdjustedtRatio'] = df_financials['revenueTTM']/df_financials['totalLiabilitiesAndStockholdersEquityAdjustedTTMAve']
    df_financials["salesToCapitalRatioFarbod"] = df_financials['revenueTTM']/df_financials['investedCapitalBookValueAdjustedTTMAve']
    df_financials["salesToCapitalRatioFarbodQ"] = df_financials['revenue']/df_financials['investedCapitalBookValueAdjusted']
    
    df_financials["ROIC"] = df_financials['operatingIncomeTTMAfterTax']/df_financials['investedCapitalBookValueDebtEquityTTMAve']
    df_financials["ROICAdjusted"] = df_financials['operatingIncomeAdjustedTTMAfterTax']/df_financials['investedCapitalBookValueDebtEquityAdjustedTTMAve']

    df_financials["ROICFarbod"] = df_financials['operatingIncomeTTMAfterTax']/df_financials['investedCapitalBookValueTTMAve']
    df_financials["ROICAdjustedFarbod"] = df_financials['operatingIncomeAdjustedTTMAfterTax']/df_financials['investedCapitalBookValueAdjustedTTMAve']

    df_financials['ROA'] = df_financials["operatingIncomeTTMAfterTax"]/(df_financials["totalLiabilitiesAndStockholdersEquityTTMAve"])
    df_financials['ROAAdjusted'] = df_financials["operatingIncomeAdjustedTTMAfterTax"]/df_financials["totalLiabilitiesAndStockholdersEquityAdjustedTTMAve"]

    df_financials["ROE"] = df_financials["netIncomeTTM"]/(df_financials["totalStockholdersEquityTTMAve"])
    df_financials["ROENetCash"] = df_financials["netIncomeTTM"]/(df_financials["totalStockholdersEquityTTMAve"]- df_financials['cashAndShortTermInvestmentsTTMAve'])

    df_financials['payOutRatio'] = (df_financials['dividendsPaid']/df_financials['netIncome'])*-1
    df_financials['payOutRatioTTM'] = (df_financials['dividendsPaidTTM']/df_financials['netIncomeTTM'])*-1
    
    df_financials['potentialPayOutRatio'] = (df_financials['dividendsPaid']/df_financials['FCFEAfterDebt'])*-1
    df_financials['potentialPayOutRatioTTM'] = (df_financials['dividendsPaid']/df_financials['FCFEAfterDebtTTM'])*-1
    #### Buffet Metrics: Moat
    # 1. Interest Expense / Operating Income < 15%
    # 2. Gross Profit / Revenue > 40%
    # 3. SG&A / Gross Profit < 30%
    # 4. Depreciation / Gross Profit < 8%
    # 5. Tax Paid / Pre-tax Income (Earnings Before Tax) ~21%
    # 6. Net Income / Revenue > 20%
    # 7. Capital Expenditures / Net Income < 8%
    # 8. Total Liabilities / Shareholder Equity < 80% 
    # 9. Net Income / Shareholder Equity > 15%


    df_financials['BuffetSG&ARatio'] = df_financials['sellingGeneralAndAdministrativeExpenses']/df_financials['grossProfit']
    df_financials['BuffetresearchAndDevelopmentExpensesRatio'] = df_financials['researchAndDevelopmentExpenses']/df_financials['grossProfit']
    df_financials['BuffetDepreciationAndAmortizationRatio'] = df_financials['depreciationAndAmortization']/df_financials['grossProfit']

    df_financials['BuffetOperatingExpensesRatio'] =df_financials['operatingExpenses']/df_financials['grossProfit']
    df_financials['BuffetOperatingExpensesAdjustedRatio']=df_financials['operatingExpensesAdjusted']/df_financials['grossProfit']

    df_financials['BuffetInterestExpenseRatio'] = df_financials['interestExpense']/df_financials['operatingIncome']
    df_financials['BuffetInterestExpenseAdjustedRatio'] = df_financials['interestExpense']/df_financials['operatingIncomeAdjusted']
    
    df_financials['BuffetTotalLiabilitiesRatio'] = df_financials['totalLiabilities']/df_financials['totalStockholdersEquity']
    df_financials['BuffetCapExRatio'] = (df_financials['capEx']*-1)/df_financials['netIncome']
    df_financials['BuffetCapExAdjustedRatio'] = (df_financials['capExAdjusted']*-1)/df_financials['netIncome']
    df_financials['averageSalesToCapitalRatio'] =(df_financials['salesToCapitalRatioFarbod'] + df_financials['salesToCapitalAdjustedRatio'])/2
    

    return(df_financials)

# df_financials['costOfRevenueLeverageRevenue'] = df_financials['costOfRevenue']/df_financials['revenue']
# df_financials['sellingGeneralAndAdministrativeExpensesLeverageRevenue'] = df_financials['sellingGenseralAndAdministrativeExpenses']/df_financials['revenue']
# df_financials['researchAndDevelopmentExpensesLeverageRevenue'] = df_financials['researchAndDevelopmentExpenses']/df_financials['revenue']
# df_financials['otherExpensesLeverageRevenue'] = df_financials['otherExpenses']/df_financials['revenue']
# df_financials['operatingExpensesLeverageRevenue'] = df_financials['operatingExpenses']/df_financials['revenue']

# df_prices_fin['sellingGeneralAndAdministrativeExpensesLeverageGrossProfit'] = df_prices_fin['sellingGeneralAndAdministrativeExpenses']/df_prices_fin['grossProfit']
# df_prices_fin['researchAndDevelopmentExpensesLeverageGrossProfit'] = df_prices_fin['researchAndDevelopmentExpenses']/df_prices_fin['grossProfit']
# df_prices_fin['otherExpensesLeverageGrossProfit'] = df_prices_fin['otherExpenses']/df_prices_fin['grossProfit']
# df_prices_fin['operatingExpensesLeverageGrossProfit'] = df_prices_fin['operatingExpenses']/df_prices_fin['grossProfit']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
def comparable_firms_finder(df_companies_profile_in_database,
                            target_company):
    """Get ticker symbols of companies in the same sector, and/or industry.
    df_companies_profile_in_database: Needs a dataframe with column symbol,industry,sector
    """
    target_company_sector = df_companies_profile_in_database[df_companies_profile_in_database['symbol']==target_company]['sector'].values[0]
    target_company_industry = df_companies_profile_in_database[df_companies_profile_in_database['symbol']==target_company]['industry'].values[0]
    ### companies_in_target_company_sector
    companies_in_target_company_sector = df_companies_profile_in_database[df_companies_profile_in_database['sector'] == target_company_sector]['symbol'].tolist()
    ### companies_in_target_company_industry
    companies_in_target_company_industry =df_companies_profile_in_database[df_companies_profile_in_database['industry'] == target_company_industry]['symbol'].tolist()
    ### companies_in_target_company_sector_and_industry
    companies_in_target_company_sector_and_industry = df_companies_profile_in_database[(df_companies_profile_in_database['sector'] == target_company_sector) &
                                                                                       (df_companies_profile_in_database['industry'] == target_company_industry)]['symbol'].tolist()
    return({"companies_in_target_company_sector":companies_in_target_company_sector,
           "companies_in_target_company_industry":companies_in_target_company_industry,
           "companies_in_target_company_sector_and_industry":companies_in_target_company_sector_and_industry})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
def date_interval_calculator(as_of_date=dt.datetime.now(),
                             interval_in_days=730):
    """as_of_date: Date of calculating from
    interval_in_days: Defualt is 730 days which the last 2 years from as_of_date
    """
    ###import datetime as dt
    beta_begining_date = as_of_date - dt.timedelta(days=interval_in_days)
    return({"as_of_date":as_of_date,
            "begining_date":beta_begining_date})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
def market_cap_provider(list_of_symbols,
                        fmp_api,
                        limit=10000000):
    """Pull historicall MarketCap"""
    list_of_market_cap=[]
    for ticker in list_of_symbols:
        try:
            list_of_market_cap.append(pd.DataFrame(fmpsdk.historical_market_capitalization(fmp_api,
                                                                                           ticker,
                                                                                           limit=limit)))
        except:
            pass
     ### Concatinate the retrieved historical marketcap
    df_market_cap = pd.concat(list_of_market_cap,
                              ignore_index=True)
    df_market_cap['Date'] = pd.to_datetime(df_market_cap['date'])
    df_market_cap.drop(['date'],axis=1,inplace=True)
    ### There might be 2 market capt for the same stock for the same date. It's data error. Drop those duplicates
    df_market_cap.drop_duplicates(['symbol','Date'],inplace=True)
    ### Resample for all days
    df_market_cap=df_market_cap.set_index("Date").groupby(
        ['symbol']).resample('1D').ffill().drop('symbol',axis=1).reset_index()
    return(df_market_cap)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
def try_function(func,
                 *args):
    """take any fucntion and pass their arguments. Try to perform the operation. If failed, return np.NaN"""
    try:
        res = func(*args)
        return(res)
    except Exception as e:
        return(np.NaN)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
def beta_calculator(x,
                    y):
    """Calcualte Beta via Fitting OLS Linear Regression Using Numpy Polyfit function"""
    ##import numpy as np
    try:
        res = np.polyfit(x,y,1)
        beta = res[0]
        alpha = res[1]
        return(beta)
    except Exception as e:
        return(np.NaN)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
def rolling_beta_calculator(df_prices,
                            market_index='SPY',
                            resample=True,
                            resample_interval='5B',
                            window=100):
    """Compute Beta via regression on rolling basis
    df_prices_of_symbols: daily price of stocks
    market_index: Market Portfolio
    window: Window of rolling regression - Number of datapoint using to compute beta"""
    def correlation_calculator(x,y):
        """Compuet R Squared"""
        correlation = np.corrcoef(x, y)[0,1]
        r_squared = correlation**2
        return(r_squared)
    def result_extractor(result,
                     variable_name='xyz'):
        """This is very specific function to be used in rolling_beta_calculator fuction"""
        dict_result = dict(result)
        df_result = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in dict_result.items()]))
        df_result = pd.melt(df_result.reset_index().rename(columns={"index":"DataPoint"}),
                                       id_vars='DataPoint',
                                       var_name='symbol',
                                       value_name=variable_name)
        return(df_result)
    df_prices_of_symbols = df_prices.copy()
    df_prices_resampled = df_prices.copy()
    if resample:
        df_prices_resampled = df_prices_resampled.set_index("Date").groupby(['symbol']).resample(
            rule=resample_interval,
            closed='right').ffill().drop('symbol',
                                         axis=1).reset_index()
    ### Compute Percent Change of symbol Price
    df_prices_resampled["Adj ClosePCT"]= df_prices_resampled.groupby(['symbol'])['Adj Close'].pct_change()
    df_prices_resampled["Adj ClosePCT USD"]= df_prices_resampled.groupby(['symbol'])['Adj Close USD'].pct_change()
    df_prices_resampled["Adj ClosePCT"+market_index]= df_prices_resampled.groupby(['symbol'])["Adj Close"+market_index].pct_change()
    df_prices_resampled.dropna(subset=['Adj ClosePCT',
                             'Adj ClosePCT USD',
                             "Adj ClosePCT"+market_index],inplace=True)
    ### Count Number of Data Points
    df_prices_resampled['DataPoint']=df_prices_resampled.groupby(['symbol']).cumcount()
    
    ### Fit Rolling Regression and compute Beta
    regression_result = df_prices_resampled.groupby(['symbol']).apply(lambda x: try_function(rolling_apply,
                                                                                   beta_calculator,
                                                                                   window,
                                                                                   x['Adj ClosePCT'+market_index],
                                                                                   x['Adj ClosePCT USD'])
                                                           )
    ### Extract the Betas from regression_result. Convert them to DataFrame and merge the results with the df_price
    df_regression_result = result_extractor(regression_result,
                                            'Beta')
    df_prices_resampled = pd.merge(df_prices_resampled,
                                   df_regression_result,
                                   left_on=['symbol','DataPoint'],
                                   right_on=['symbol','DataPoint'],
                                   how='left')
    
    ### Fit Rolling rolling correlation to get r_squared
    r_squared_result = df_prices_resampled.groupby(['symbol']).apply(lambda x: try_function(rolling_apply,
                                                                                   correlation_calculator,
                                                                                   window,
                                                                                   x['Adj ClosePCT'+market_index],
                                                                                   x['Adj ClosePCT USD'])
                                                           )
    df_r_squared_result = result_extractor(r_squared_result,'RSquared')
    df_prices_resampled = pd.merge(df_prices_resampled,
                                   df_r_squared_result,
                                   left_on=['symbol','DataPoint'],
                                   right_on=['symbol','DataPoint'],
                                   how='left')

    
    df_prices_resampled = df_prices_resampled[['Date','symbol','Beta','RSquared']].copy()
    ### Resample againe to merge betas with orignal price dataset
    df_prices_resampled = df_prices_resampled.set_index("Date").groupby(['symbol']).resample(
        rule='1D',
        closed='right').ffill().drop('symbol',
                                     axis=1).reset_index()
    df_prices_of_symbols = pd.merge(df_prices_of_symbols,
                         df_prices_resampled,
                         left_on=['Date','symbol'],
                         right_on=['Date','symbol'],
                         how='left')
    return(df_prices_of_symbols)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
def yahoo_quotes_provider(list_of_symbols):
    """Get Current quote from Yahoo Finance for a list of symbols/tciker"""
    list_of_df_quotes=[]
    for symbol in list_of_symbols:
        try:
            df_quote_yahoo_symbol = pandas_datareader.data.get_quote_yahoo(symbol)
            df_quote_yahoo_symbol['symbol']=symbol
            list_of_df_quotes.append(df_quote_yahoo_symbol)
        except:
            pass
    df_quotes = pd.concat(list_of_df_quotes,
                          ignore_index=True)
    return(df_quotes)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
def historical_ticker_symbol_price_provider(list_of_symbols,
                                            fmp_api):
    set_of_symbols = set(list_of_symbols)
    #### If there is only on ticker to retrrieved
    if len(list_of_symbols)==1:
        df_full_prices_melt = yf.download(list_of_symbols,
                                          period="max")
        df_full_prices_melt.reset_index(inplace=True)
        df_full_prices_melt['symbol']=list_of_symbols[0]
        
        df_full_prices_melt=df_full_prices_melt[['Date', 'symbol', 'Adj Close', 
                                                 'Close', 'High', 'Low', 'Open',
                                                 'Volume']].copy()
        
        return(df_full_prices_melt)
    ### if there are more than 1 ticker to be called 
    df_full_prices = yf.download(list_of_symbols,
                                 period="max")
    df_full_prices_melt= pd.melt(df_full_prices.reset_index(),
                                id_vars='Date')
    df_full_prices_melt = df_full_prices_melt.dropna()
    df_full_prices_melt.reset_index(drop=True,inplace=True)
    df_full_prices_melt= pd.pivot(df_full_prices_melt,
                                  index=['Date','variable_1'],
                                  columns=['variable_0']).reset_index()
    df_full_prices_melt= data_frame_flattener(df_full_prices_melt)
    df_full_prices_melt.rename(columns={"variable_1":"symbol",
                                        "value Adj Close":"Adj Close",
                                        "value Open":"Open",
                                        "value Close":"Close",
                                        "value High":"High",
                                        "value Low":"Low",
                                        "value Volume":"Volume"},
                               inplace=True)
        
    ### Get prices of symbols which were not available in yahoo finance
    set_of_symbol_with_prices_from_yahoo_finance = set(df_full_prices_melt["symbol"])
    set_of_symbols_not_in_yahoo = set_of_symbols - set_of_symbol_with_prices_from_yahoo_finance
    list_of_df_prices_from_fmp=[]
    for ticker in set_of_symbols_not_in_yahoo:
        try:
            df_ticker_price_fmp = pd.DataFrame(fmpsdk.historical_price_full(fmp_api,
                                                                            ticker,
                                                                            from_date='1960-01-01'))
            df_ticker_price_fmp['symbol'] = ticker
            if len(df_ticker_price_fmp)>0:
                list_of_df_prices_from_fmp.append(df_ticker_price_fmp)
        except:
            pass
    ### Make the Prices from FMP consitant with the format of prices form yahoo
    if len(list_of_df_prices_from_fmp)>0:
        print(len(list_of_df_prices_from_fmp))
        df_price_fmp = pd.concat(list_of_df_prices_from_fmp,
                                 ignore_index=True)
        df_price_fmp['Date'] = pd.to_datetime(df_price_fmp['date'])
        df_price_fmp.rename(columns={"adjClose":"Adj Close",
                                     "close":"Close",
                                     "low":"Low",
                                     "high":"High",
                                     "open":"Open",
                                     "volume":"Volume"},inplace=True)
        df_price_fmp = df_price_fmp[['Date', 'symbol', 'Adj Close', 
                                     'Close', 'High', 'Low', 'Open','Volume']].copy()
        ### Concate Prices from Yahoo and FMP
        df_full_prices_melt = pd.concat([df_price_fmp,
                                         df_full_prices_melt],
                                        ignore_index=True)
    print("Getting Current quote from Yahoo Finance")
    ### Get current number of shares out standing and marketcap from Yahoo Finance
    df_yahoo_quotes = yahoo_quotes_provider(list_of_symbols)
    df_yahoo_quotes = df_yahoo_quotes[['symbol','currency','sharesOutstanding','marketCap']].copy()
    df_yahoo_quotes.rename(columns={"currency":"symbolCurrency"},inplace=True)
    df_yahoo_quotes= column_suffix_adder(df_yahoo_quotes,
                                                    ['sharesOutstanding'
                                                     ,'marketCap'],
                                                    'CurrentYahoo')
    df_full_prices_melt = pd.merge(df_full_prices_melt,
                                   df_yahoo_quotes,
                                   left_on=['symbol'],
                                   right_on=['symbol'],
                                   how='left')
    ### Conventional Market Cap
    df_full_prices_melt['marketCapConventional']=df_full_prices_melt['Close']*df_full_prices_melt['sharesOutstandingCurrentYahoo']
    return(df_full_prices_melt)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
def treasury_yeild_provider(symbol='^TNX'):
    """Treausury Yeild From Yahoo Finance - Risk Free Rate"""
    df_rf = historical_ticker_symbol_price_provider([symbol],
                                                "NONE")
    ### Add current datetime in case if there is no rate available
    df_rf = pd.concat([df_rf,
                       pd.DataFrame([{'Date':dt.datetime.now()}])],ignore_index=True)
    df_rf.drop_duplicates("Date",keep='first',inplace=True)
    df_rf=df_rf.set_index("Date").resample('1D').ffill().reset_index()
    df_rf.rename(columns={"Adj Close":"10YearRF"},inplace=True)
    df_rf=df_rf[['Date','10YearRF']].copy()
    df_rf['10YearRF']=df_rf['10YearRF']/100
    return(df_rf)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [72]:
def historical_ticker_symbol_and_market_index_return_provider(list_of_symbols,
                                                              market_index='SPY',
                                                              resample=False,
                                                              resample_interval='5B',
                                                              window=50,
                                                              market_cap=False,
                                                              df_companies_profile_in_database= None,
                                                              fmp_api="your_fmp_api_key",
                                                              fred_api="your_fred_api_key"):
    """
    Today Market cap is not avaialble until end of the day. So I am using previus avaialble marketcap. Should correct it 
    Using Yahoo Finance API pandas_data_rdae_get_qouate_yahoo
    df_companies_profile_in_database:Needs a dataframe with columns named ['symbol','currency','industry','sector','companyName','country'], if it's not provded, they are retreived from FMP
    list_of_symbols: list of ticker to pull their prices
    market_index: market index to pull its hitorical prices. Default is SPY
    resample: resmaple return (1Day,2Day,3Dday...ND). To go to weeekly rerurn from Daily Return set resample_interval='5B'
    window: Number of widnow size to use for computing Beta. If you resample 5B and choose window of 50, regression beta would be 
    based 50 datapoint and each data point is based on 5 Bussiness day return. 
    The total length of beta would be 50*5 = 250 business days which translate to 1 calandar year.
    """
    ### Loading companies profile
    if df_companies_profile_in_database is None:
        print("loading companies profile from FMP")
        df_companies_profile_in_database = companies_profile_retriever(list_of_symbol = list_of_symbols,
                                                                       fmp_api = fmp_api)
    ### Exchnage Rates
    print("Getting currency exchange ticker symbol adj close price")
    df_currency_ex_rate = historical_exchage_rate_provider()
    ### Ticker Prices
    print("Getting ticker symbol adj close price")
    df_prices = historical_ticker_symbol_price_provider(list_of_symbols,
                                                       fmp_api =fmp_api)
    df_prices.drop(['High','Low','Open','Volume'],axis=1,inplace=True)

    
    ### Market Index Prices
    print("Getting market ticker symbol adj close price")
    df_market= historical_ticker_symbol_price_provider([market_index],
                                                       fmp_api = fmp_api)
    df_market.drop(['High','Low','Open','Volume'],axis=1,inplace=True)
    
    print("Price retrieval Complete")
    ### Get Sector, Industry, Company Name and counrty of each ticker
    df_prices= pd.merge(df_prices,
                        df_companies_profile_in_database[['symbol','currency','industry',
                                                          'sector','companyName','country']].drop_duplicates("symbol"),
                        left_on='symbol',
                        right_on='symbol',
                        how='left')
    
    ### Merge Ticker symbol prices with the market index
    df_prices = pd.merge(df_prices,
                         df_market.drop('symbol',axis=1),
                         left_on='Date',
                         right_on='Date',
                         how='outer',
                        suffixes=("",market_index))
    
    ### Drop Missing Prices
    df_prices = df_prices.dropna(subset=['Adj Close', "Adj Close"+market_index])
    df_prices= df_prices.sort_values(['symbol','Date']).reset_index(drop=True)
    ### Merge Exchange Rates with Stock Prices
    df_prices= pd.merge(df_prices,
                        df_currency_ex_rate,
                        left_on=['Date','currency'],
                        right_on=['Date','currency'],
                        how="left")
    
    ### If there is no currency rate, fill the rate with 1
    df_prices['currency_rate'] = df_prices['currency_rate'].fillna(1)
    
    ### Convert price of stock to USD
    df_prices['Adj Close USD']= df_prices['Adj Close']/df_prices['currency_rate']
    df_prices['Close USD']= df_prices['Close']/df_prices['currency_rate']
    
    print("Computing Beta")
    df_prices = rolling_beta_calculator(df_prices,
                                        market_index=market_index,
                                        resample=resample,
                                        resample_interval=resample_interval,
                                        window=window)
    ### Get Risk Free Rate
    print("Getting Risk Free Rake")
    df_rf = treasury_yeild_provider(symbol='^TNX')
    df_prices = pd.merge(df_prices,
                         df_rf,
                         left_on=['Date'],
                         right_on=['Date'],
                         how='left')
    print("Getting Macro Factors")
    df_factor_from_yahoo = yf.download(['GC=F','CL=F'],
                                   period="max")['Adj Close'].ffill().resample("D").ffill()
    df_factor_from_yahoo = pd.merge(df_factor_from_yahoo, 
                                    column_suffix_adder(df_factor_from_yahoo.rolling(90).mean().reset_index(),
                                                                   list_of_columns_to_add_suffix_on=['CL=F','GC=F'],
                                                                   suffix='_3M_Ave'),
                                    left_on=['Date'],
                                    right_on='Date',
                                    how='left')
    
    df_macro_from_fred  = pd.pivot(data_retriver_from_fred(['DBAA','DAAA','GDP','GDPC1','CPIAUCSL','WM2NS','M2SL'],
                                                           start_date=dt.datetime(1950,1,1),
                                                           fred_api=fred_api),
                                   columns=['id'],
                                   index='Date',
                                   values='value').ffill().resample("D").ffill().reset_index()
    df_macro_from_fred['DBAA'] = df_macro_from_fred['DBAA']/100
    df_macro_from_fred['DAAA'] = df_macro_from_fred['DAAA']/100
                               
    df_macros = pd.merge(df_factor_from_yahoo,
                         df_macro_from_fred,
                         left_on='Date',
                         right_on='Date',
                         how='outer').sort_values("Date").ffill()

    df_prices = pd.merge(df_prices,
                         df_macros,
                         left_on=['Date'],
                         right_on=['Date'],
                         how='left')

    ### Get Market Cap of tickers
    if market_cap:
        try:
            print("Getting Historical MarketCap")
            df_market_cap = market_cap_provider(list_of_symbols,
                                                fmp_api=fmp_api,
                                                limit=10000000)
            ### Merge Market Cap with Prices
            df_prices = pd.merge(df_prices,
                                df_market_cap,
                                left_on=['symbol','Date'],
                                right_on=['symbol','Date'],
                                how="left")
            ### This is to fill current Marekt Cap with previous data point. Should be corrected.
            df_prices["marketCap"] = df_prices.groupby(["symbol"])['marketCap'].ffill()
            df_prices['marketCapUSD']= df_prices['marketCap']/df_prices['currency_rate']
            df_prices['marketCapUSDConventional']= df_prices['marketCapConventional']/df_prices['currency_rate']
        except:
            pass
    return({"prices":df_prices,
            "currency_ex_rate":df_currency_ex_rate})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [73]:
def historical_price_and_financials(list_of_symbols,
                                    market_index='SPY',
                                    resample=False,
                                    resample_interval='5B',
                                    window=50,
                                    market_cap=True,
                                    rnd_amortization_period_begin=0,
                                    rnd_amortization_period=3,
                                    fmp_api="your_fmp_api_key",
                                    fred_api="your_fred_api_key",
                                    df_companies_profile_in_database = None):
    """Return stock price, marketcap, return and companies financials"""
    df_financials = pd.DataFrame()
    ### Store output of historical_ticker_symbol_and_market_index_return_provider
    ticker_symbol_and_market_index = historical_ticker_symbol_and_market_index_return_provider(
        df_companies_profile_in_database = df_companies_profile_in_database,
        list_of_symbols=list_of_symbols,
        market_index=market_index,
        resample=resample,
        resample_interval=resample_interval,
        window=window,
        market_cap=market_cap,
        fmp_api=fmp_api,
        fred_api=fred_api)
    ### Get Historical Prices
    df_prices = ticker_symbol_and_market_index["prices"].copy()
    ### Get Historical Currency Exchnage Rate to USD
    df_currency_ex_rate_from_ticker_symbol_and_market_index_function = ticker_symbol_and_market_index["currency_ex_rate"].copy()
    ### Add Marginal Tax Rate
    print("Reading Tax Rate")
    #df_marginal_tax_rate = pd.read_excel(path_of_data+"Tax Rates/Corporate Marginal Rates.xlsx")
    url_marginal_tax_rate = "https://docs.google.com/spreadsheets/d/156a9vyK3Q01ApSypU_0x4UOagejE5ofK"
    df_marginal_tax_rate = pd.read_excel(url_marginal_tax_rate+'/export')
    
    df_prices['Year']=df_prices['Date'].dt.year
    df_prices= pd.merge(df_prices,
                        df_marginal_tax_rate,
                        left_on=['Year'],
                        right_on=['Year'],
                        how='left')
    print("Reading Tax Rate Complete")
    list_of_df_financials=[]
    print("Getting Historical Financial Statements")
    for ticker in list_of_symbols:
        try:
            df_ticker_financials = financial_statments_preparator(ticker,
                                                                  fmp_api=fmp_api,
                                                                  period='quarter',
                                                                  rnd_amortization_period_begin= rnd_amortization_period_begin,
                                                                  rnd_amortization_period=rnd_amortization_period,
                                                                 convert_to_USD=True,
                                                                 df_currency_ex_rate=df_currency_ex_rate_from_ticker_symbol_and_market_index_function)
            list_of_df_financials.append(df_ticker_financials)
        except:
            pass
    print(len(list_of_df_financials))
    if len(list_of_df_financials)>0:
        df_financials = pd.concat(list_of_df_financials,
                                  ignore_index=True)
        df_financials.rename(columns={"date":"dateFinncialEnding"},
                             inplace=True)
        df_financials['fillingDate_IS']=pd.to_datetime(df_financials['fillingDate_IS'])
        df_financials['fillingDate_BS']=pd.to_datetime(df_financials['fillingDate_BS'])
        df_financials['fillingDate']=pd.to_datetime(df_financials['fillingDate'])
        df_financials['Date']= pd.to_datetime(df_financials['fillingDate_IS'])
        ### Resample financials for every day
        df_financials_resampled = df_financials.set_index("Date").groupby(
            ['symbol']).resample('1D').ffill().drop(['symbol'],axis=1).reset_index()
        ### Get the columns to ffill
        list_of_col_from_financials_to_ffill= list(set(df_financials_resampled.columns) -set(df_prices.columns))
        ### Merge Prices with financials
        ### Finacials release date is always after the date of finaicials
        df_prices_and_financials = pd.merge(df_prices,
                                           df_financials_resampled.drop(['currency_rate'],
                                                                        axis=1),
                                           left_on=['Date','symbol'],
                                           right_on=['Date','symbol'],
                                           how="left")
        ### ffill columns in df_prices_and_financials
        df_prices_and_financials[list_of_col_from_financials_to_ffill]= df_prices_and_financials.groupby(["symbol"])[
            list_of_col_from_financials_to_ffill].ffill()
        #### Debt to Equity Ratio. I am using marketcap USD from fmp
        df_prices_and_financials['debtEquityRatio'] = df_prices_and_financials['totalDebt']/df_prices_and_financials['marketCapUSD']
        ############### Compute Industry Avegrages Metrics   ########################
        list_of_meterics=['Beta','RSquared','debtEquityRatio','cashToAsset',
                          'grossProfitRatio','grossProfitRatioTTM',
                          'operatingIncomeRatioTTM',
                          'operatingIncomeAdjustedRatioTTM',
                          'netIncomeRatioTTM',
                          'netIncomeRatio',
                          'operatingIncomeRatio',
                          'operatingIncomeAdjustedRatio',
                          'frimReinvestmentRateTTM',
                          'equityReinvestmentRateTTM',
                          'salesToAssetRatio','salesToAssetAdjustedtRatio',
                           'salesToCapitalRatio','salesToCapitalAdjustedRatio',
                          'salesToCapitalRatioFarbod','salesToCapitalRatioFarbodQ',
                           'ROIC','ROICAdjusted',
                          'ROICFarbod',
                          'currentRatio','longTermRatio','debtToAsset',
                          'variabilitySG&A','operatingLeverage','operatingLeverageAdjusted','operatingLeverageYoY',
                          'ROICAdjustedFarbod',
                          'ROA','ROAAdjusted',
                          'ROE','ROENetCash',
                          'BuffetresearchAndDevelopmentExpensesRatio',
                          'BuffetOperatingExpensesRatio','BuffetOperatingExpensesAdjustedRatio',
                          'BuffetSG&ARatio','BuffetInterestExpenseRatio','BuffetInterestExpenseAdjustedRatio',
                          'BuffetDepreciationAndAmortizationRatio','BuffetTotalLiabilitiesRatio',
                          'BuffetCapExRatio','BuffetCapExAdjustedRatio']

        df_prices_and_financials['cashToAsset']=df_prices_and_financials['cashAndCashEquivalents']/df_prices_and_financials['totalLiabilitiesAndStockholdersEquity']
        ### Compute Median of meterics by industry
        df_prices_and_financials = pd.merge(df_prices_and_financials,
                                            column_suffix_adder(df_prices_and_financials.groupby(['Date','industry'])[list_of_meterics].median().reset_index(),
                                                                           list_of_meterics,
                                                                           "IndustryMedian"),
                                            left_on=['Date','industry'],
                                            right_on=['Date','industry'],
                                            how='left')
        ### Compute Median of meterics by Sector
        df_prices_and_financials = pd.merge(df_prices_and_financials,
                                            column_suffix_adder(
                                                df_prices_and_financials.groupby(['Date','sector'])[
                                                    list_of_meterics].median().reset_index(),
                                                list_of_meterics,
                                                "SectorMedian"),
                                            left_on=['Date','sector'],
                                            right_on=['Date','sector'],
                                            how='left')
        ###Compute Median of meterics by Industry-Sector
        df_prices_and_financials = pd.merge(df_prices_and_financials,
                         column_suffix_adder(df_prices_and_financials.groupby(['Date','industry','sector'])[list_of_meterics].median().reset_index(),
                                                        list_of_meterics,
                                                   "IndustrySectorMedian"),
                                            left_on=['Date','industry','sector'],
                                            right_on=['Date','industry','sector'],
                                            how='left')
        
        ###Compute Median of meterics by sample
        df_prices_and_financials = pd.merge(df_prices_and_financials,
                         column_suffix_adder(df_prices_and_financials.groupby(['Date'])[list_of_meterics].median().reset_index(),
                                                        list_of_meterics,
                                                   "SampleMedian"),
                                            left_on=['Date'],
                                            right_on=['Date'],
                                            how='left')
        # Unlever the beta for each day by median industry beta and clean up for cash
        # Unlevered beta = Regression beta / (1 + (1-tax rate) Debt/Equity)
        # Unlevered beta cleaned for cash = Unlevered beta/(1-(cash/asset))
        # t: is the marginal tax rate
        df_prices_and_financials['unleveredBetaIndustryMedian']= ((df_prices_and_financials['BetaIndustryMedian']/
                                                                  (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                   df_prices_and_financials['debtEquityRatioIndustryMedian']))/(1-(df_prices_and_financials["cashToAssetIndustryMedian"])))
        df_prices_and_financials['unleveredBetaSectorMedian']= ((df_prices_and_financials['BetaSectorMedian']/
                                                                  (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                   df_prices_and_financials['debtEquityRatioSectorMedian']))/(1-(df_prices_and_financials["cashToAssetSectorMedian"])))
        df_prices_and_financials['unleveredBetaIndustrySectorMedian']= ((df_prices_and_financials['BetaIndustrySectorMedian']/
                                                                        (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                         df_prices_and_financials['debtEquityRatioIndustrySectorMedian']))/(1-(df_prices_and_financials["cashToAssetIndustrySectorMedian"])))
        
        df_prices_and_financials['unleveredBetaSampleMedian']= ((df_prices_and_financials['BetaSampleMedian']/
                                                                        (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                         df_prices_and_financials['debtEquityRatioSampleMedian']))/(1-(df_prices_and_financials["cashToAssetSampleMedian"])))
        
        ### Company Levered Beta  = Unlevered beta * (1 + (1- tax rate) (Debt/Equity))
        df_prices_and_financials['companyBetaIndustry'] = (df_prices_and_financials['unleveredBetaIndustryMedian']*
                                                           (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                            df_prices_and_financials['debtEquityRatio']))
        df_prices_and_financials['companyBetaSector'] = (df_prices_and_financials['unleveredBetaSectorMedian']*
                                                         (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                          df_prices_and_financials['debtEquityRatio']))
        df_prices_and_financials['companyBetaIndustrySector'] = (df_prices_and_financials['unleveredBetaIndustrySectorMedian']*
                                                                (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                 df_prices_and_financials['debtEquityRatio']))
        df_prices_and_financials['companyBetaSample'] = (df_prices_and_financials['unleveredBetaSampleMedian']*
                                                                (1+(1-df_prices_and_financials['marginalTaxRate'])*
                                                                 df_prices_and_financials['debtEquityRatio']))
        #df_prices_and_financials['companyBeta'] = (df_prices_and_financials['companyBetaIndustry']+df_prices_and_financials['companyBetaSector'])/2
        #################### End of Metrics Computation #################
        df_prices_and_financials['fiancialEndingYear'] = df_prices_and_financials['dateFinncialEnding'].dt.year
        return({"financials":df_financials,
                "prices_and_financials":df_prices_and_financials,
                "currency_ex_rate":df_currency_ex_rate_from_ticker_symbol_and_market_index_function})
    else:
        return({"prices":df_prices,
                "currency_ex_rate":df_currency_ex_rate_from_ticker_symbol_and_market_index_function})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
def dynamic_converger(current,
                      expected,
                      number_of_steps,
                      period_to_begin_to_converge):
    """This Function is to project growth in 2 phase.
    Phase 1: You grow  Period after period for number of period specified.
    Phase 2: growth begin to converge to number_of_steps value.
    current: begining growth_rate
    expected: final growth rate
    period_to_begin_to_converge: Period to begin to transition to terminal growth value 
    number_of_steps: number of period (years) to project growth."""
    number_of_steps =  int(number_of_steps)
    period_to_begin_to_converge = int(period_to_begin_to_converge)
    def converger(current,
              expected,
              number_of_steps):
        list_of_values=[]
        list_of_values.append(current)
        for iteration in np.arange(0,number_of_steps):
            value = (current)-((current-expected)/number_of_steps)*(iteration+1)
            list_of_values.append(value)
        return(pd.Series(list_of_values,dtype=float))
    
    phase1= pd.Series([current]*(period_to_begin_to_converge-1),
                     dtype=float)
    phase2=converger(current,
                     expected,
                     number_of_steps-period_to_begin_to_converge)
    result= pd.concat([phase1,phase2],ignore_index=True)
    return(result)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
def dynamic_converger_multiple_phase(growth_rates_for_each_cylce,
                                     length_of_each_cylce,
                                     convergance_periods):
    list_of_results = []
    for cycle in range(len(length_of_each_cylce)):
        result = dynamic_converger(current = growth_rates_for_each_cylce[cycle][0],
                        expected = growth_rates_for_each_cylce[cycle][1],
                        number_of_steps = length_of_each_cylce[cycle],
                        period_to_begin_to_converge = convergance_periods[cycle])
        list_of_results.append(result)
    return(pd.concat(list_of_results,ignore_index=True))


def revenue_projector_multi_phase(revenue_base,
                                  revenue_growth_rate_cycle1_begin,
                                  revenue_growth_rate_cycle1_end,
                                  revenue_growth_rate_cycle2_begin,
                                  revenue_growth_rate_cycle2_end,
                                  revenue_growth_rate_cycle3_begin,
                                  revenue_growth_rate_cycle3_end = 0.028,
                                  length_of_cylcle1=3,
                                  length_of_cylcle2=4,
                                  length_of_cylcle3=3,
                                  revenue_convergance_periods_cycle1 =1,
                                  revenue_convergance_periods_cycle2=1,
                                  revenue_convergance_periods_cycle3=1):
    projected_revenue_growth = dynamic_converger_multiple_phase(growth_rates_for_each_cylce= [[revenue_growth_rate_cycle1_begin,revenue_growth_rate_cycle1_end],
                                                               [revenue_growth_rate_cycle2_begin,revenue_growth_rate_cycle2_end],
                                                               [revenue_growth_rate_cycle3_begin,revenue_growth_rate_cycle3_end]],
                                     length_of_each_cylce=[length_of_cylcle1,length_of_cylcle2,length_of_cylcle3],
                                     convergance_periods=[revenue_convergance_periods_cycle1,
                                                          revenue_convergance_periods_cycle2,
                                                          revenue_convergance_periods_cycle3])
    ### Compute Cummulative revenue_growth
    projected_revenue_growth_cumulative = (1+projected_revenue_growth).cumprod()
    projected_revneues = revenue_base*projected_revenue_growth_cumulative
    return(projected_revneues,projected_revenue_growth)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
def operating_margin_projector(current_operating_margin,
                               terminal_operating_margin,
                               valuation_interval_in_years=10,
                               year_operating_margin_begins_to_converge_to_terminal_operating_margin=5):
    projectd_operating_margin = dynamic_converger(current_operating_margin,
                                                  terminal_operating_margin,
                                                  valuation_interval_in_years,
                                                  year_operating_margin_begins_to_converge_to_terminal_operating_margin)
    return(projectd_operating_margin)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [77]:
def tax_rate_projector(current_effective_tax_rate,
                      marginal_tax_rate,
                      valuation_interval_in_years=10,
                      year_effective_tax_rate_begin_to_converge_marginal_tax_rate=5):
    """Project tax rate during valuation Cylce"""
    projected_tax_rate = dynamic_converger(current_effective_tax_rate,
                                           marginal_tax_rate,
                                           valuation_interval_in_years,
                                           year_effective_tax_rate_begin_to_converge_marginal_tax_rate)
    return(projected_tax_rate)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
def cost_of_capital_projector(unlevered_beta,
                              terminal_unlevered_beta,
                              current_pretax_cost_of_debt,
                              terminal_pretax_cost_of_debt,
                              equity_value,
                              debt_value,
                              marginal_tax_rate=.21,
                              risk_free_rate=0.015,
                              ERP=0.055,
                              valuation_interval_in_years=10,
                              year_beta_begins_to_converge_to_terminal_beta=5,
                              year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt=5):
    """Project Cost of Capiatal during valuation Cylce"""
    ### Compute Beta During Valuatio Cycle
    ### Company Levered Beta  = Unlevered beta * (1 + (1- tax rate) (Debt/Equity))
    company_beta = unlevered_beta * (1+(1-marginal_tax_rate)*(debt_value/equity_value))
    terminal_beta = terminal_unlevered_beta * (1+(1-marginal_tax_rate)*(debt_value/equity_value))
    beta_druing_valution_cycle = dynamic_converger(company_beta,
                                                   terminal_beta,
                                                   valuation_interval_in_years,
                                                   year_beta_begins_to_converge_to_terminal_beta)
    ### Compute Pre Tax Cost Of debt During Valuation Cycle
    pre_tax_cost_of_debt_during_valution_cycle = dynamic_converger(current_pretax_cost_of_debt,
                                                                   terminal_pretax_cost_of_debt,
                                                                   valuation_interval_in_years,
                                                                   year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt)
    
    total_capital = equity_value+debt_value
    equity_to_capital = equity_value/total_capital
    debt_to_capital = debt_value/total_capital
    after_tax_cost_of_debt_during_valution_cycle = pre_tax_cost_of_debt_during_valution_cycle*(1-marginal_tax_rate)
    cost_of_equity = risk_free_rate + (beta_druing_valution_cycle*ERP)
    cost_of_capital_during_valuatio_cycle = ((equity_to_capital*cost_of_equity)+
                                             (debt_to_capital*after_tax_cost_of_debt_during_valution_cycle))
    return(cost_of_capital_during_valuatio_cycle,beta_druing_valution_cycle,terminal_beta,cost_of_equity)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
def revenue_growth_projector(revenue_growth_rate,
                             terminal_growth_rate=.028,
                             valuation_interval_in_years=10,
                             year_revenue_growth_begin_to_converge_to_terminal_growth_rate = 5):
    """Project revenue growth during valuation Cylce"""
    projected_revenue_growth = dynamic_converger(revenue_growth_rate,
                                                 terminal_growth_rate,
                                                 valuation_interval_in_years,
                                                 year_revenue_growth_begin_to_converge_to_terminal_growth_rate)
    return(projected_revenue_growth)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [80]:
def revenue_projector(revenue_base,
                      revenue_growth_rate,
                      terminal_growth_rate,
                      valuation_interval_in_years,
                      year_revenue_growth_begin_to_converge_to_terminal_growth_rate):
    ### Estimate Revenue Growth
    projected_revenue_growth = revenue_growth_projector(revenue_growth_rate=revenue_growth_rate,
                                                        terminal_growth_rate = terminal_growth_rate,
                                                        valuation_interval_in_years=valuation_interval_in_years,
                                                        year_revenue_growth_begin_to_converge_to_terminal_growth_rate=year_revenue_growth_begin_to_converge_to_terminal_growth_rate)
    ### Compute Cummulative revenue_growth
    projected_revenue_growth_cumulative = (1+projected_revenue_growth).cumprod()
    projected_revneues = revenue_base*projected_revenue_growth_cumulative
    return(projected_revneues,projected_revenue_growth)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [81]:
def sales_to_capital_projector(current_sales_to_capital_ratio,
                               terminal_sales_to_capital_ratio,
                               valuation_interval_in_years=10,
                               year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital=3):
    projectd_sales_to_capiatl = dynamic_converger(current_sales_to_capital_ratio,
                                                  terminal_sales_to_capital_ratio,
                                                  valuation_interval_in_years,
                                                  year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital)
    return(projectd_sales_to_capiatl)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
def reinvestment_projector(revenue_base,
                           projected_revneues,
                           sales_to_capital_ratios,
                           asset_liquidation_during_negative_growth=0):
    reinvestment = (pd.concat([pd.Series(revenue_base),
                               projected_revneues],
                             ignore_index=False).diff().dropna()/sales_to_capital_ratios)
    reinvestment = reinvestment.where(reinvestment>0, (reinvestment*asset_liquidation_during_negative_growth))
    return(reinvestment)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Valuator

In [83]:
def valuator(unlevered_beta,
            terminal_unlevered_beta,
            current_pretax_cost_of_debt,
            terminal_pretax_cost_of_debt,
            equity_value,
            debt_value,
            marginal_tax_rate,
            risk_free_rate,
            ERP,
            valuation_interval_in_years,
             year_beta_begins_to_converge_to_terminal_beta,
             year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
             revenue_growth_rate,
             terminal_growth_rate,
             revenue_base,
            year_revenue_growth_begin_to_converge_to_terminal_growth_rate,
            current_effective_tax_rate,
            year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
            current_sales_to_capital_ratio,
            terminal_sales_to_capital_ratio,
            year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital,
            current_operating_margin,
            terminal_operating_margin,
            year_operating_margin_begins_to_converge_to_terminal_operating_margin,
            additional_return_on_cost_of_capital_in_perpetuity=0.0,
            cash_and_non_operating_asset=0.0,
             asset_liquidation_during_negative_growth=0):
    ### Estimate Cost of Capital during the valution cycle
    projected_cost_of_capital, projected_beta , terminal_beta, projected_cost_of_equity = cost_of_capital_projector(unlevered_beta=unlevered_beta,
                                                          terminal_unlevered_beta = terminal_unlevered_beta,
                                                          current_pretax_cost_of_debt=current_pretax_cost_of_debt,
                                                          terminal_pretax_cost_of_debt=terminal_pretax_cost_of_debt,
                                                          equity_value=equity_value,
                                                          debt_value=debt_value,
                                                          marginal_tax_rate=marginal_tax_rate,
                                                          risk_free_rate=risk_free_rate,
                                                          ERP=ERP,
                                                          valuation_interval_in_years=valuation_interval_in_years,
                                                          year_beta_begins_to_converge_to_terminal_beta=year_beta_begins_to_converge_to_terminal_beta,
                                                          year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt=year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt)
    projected_cost_of_capital_cumulative= (1+projected_cost_of_capital).cumprod()
    projected_cost_of_equity_cumulative= (1+projected_cost_of_equity).cumprod()
    ### Estimate Future Revenues
    projected_revneues,projected_revenue_growth = revenue_projector(revenue_base=revenue_base,
                                           revenue_growth_rate=revenue_growth_rate,
                                           terminal_growth_rate=terminal_growth_rate,
                                           valuation_interval_in_years=valuation_interval_in_years,
                                           year_revenue_growth_begin_to_converge_to_terminal_growth_rate=year_revenue_growth_begin_to_converge_to_terminal_growth_rate)
    ### Estmimate tax rates
    projected_tax_rates = tax_rate_projector(current_effective_tax_rate=current_effective_tax_rate,
                                            marginal_tax_rate=marginal_tax_rate,
                                            valuation_interval_in_years=valuation_interval_in_years,
                                            year_effective_tax_rate_begin_to_converge_marginal_tax_rate=year_effective_tax_rate_begin_to_converge_marginal_tax_rate)
    ### Estimate slaes to capital ratio during valuation for reinvestment
    sales_to_capital_ratios = sales_to_capital_projector(current_sales_to_capital_ratio,
                               terminal_sales_to_capital_ratio,
                               valuation_interval_in_years=valuation_interval_in_years,
                               year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital=year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital)

    ### Estimate Reinvestemnt
    projected_reinvestment = reinvestment_projector(revenue_base=revenue_base,
                                                    projected_revneues = projected_revneues,
                                                    sales_to_capital_ratios=sales_to_capital_ratios,
                                                    asset_liquidation_during_negative_growth=asset_liquidation_during_negative_growth)
    ### Operating Margin
    projected_operating_margins = operating_margin_projector(current_operating_margin,
                                                            terminal_operating_margin,
                                                            valuation_interval_in_years=valuation_interval_in_years,
                                                            year_operating_margin_begins_to_converge_to_terminal_operating_margin=year_operating_margin_begins_to_converge_to_terminal_operating_margin)
    ###EBIT
    projected_operating_income = projected_revneues * projected_operating_margins
    ### After Tax EBIT (EBI)
    projected_operating_income_after_tax = (projected_operating_income*(1-projected_tax_rates))
    ### FCFF: EBI-Reinvestment
    projected_FCFF = projected_operating_income_after_tax - projected_reinvestment
    
    ### Compute Terminal Value
    terminal_cost_of_capital = projected_cost_of_capital[-1:].values
    terminal_cost_of_equity = projected_cost_of_equity[-1:].values
    if terminal_growth_rate < 0:
        terminal_reinvestment_rate=0
    else:
        terminal_reinvestment_rate = terminal_growth_rate/(terminal_cost_of_capital+additional_return_on_cost_of_capital_in_perpetuity)
    terminal_revenue = projected_revneues[-1:].values * (1+terminal_growth_rate)
    terminal_operating_income = terminal_revenue * terminal_operating_margin
    terminal_operating_income_after_tax = terminal_operating_income*(1-marginal_tax_rate)
    terminal_reinvestment = terminal_operating_income_after_tax* terminal_reinvestment_rate
    terminal_FCFF = terminal_operating_income_after_tax - terminal_reinvestment
    termimal_discount_rate = (terminal_cost_of_capital-terminal_growth_rate)*(1+projected_cost_of_capital).prod()
    termimal_equity_discount_rate = (terminal_cost_of_equity-terminal_growth_rate)*(1+projected_cost_of_equity).prod()

    ### Concatinate Projected Values with termianl values
    projected_cost_of_capital_cumulative = pd.concat([projected_cost_of_capital_cumulative,
                                                    pd.Series(termimal_discount_rate)])
    projected_cost_of_equity_cumulative = pd.concat([projected_cost_of_equity_cumulative,
                                                    pd.Series(termimal_equity_discount_rate)])
    projected_revenue_growth = pd.concat([projected_revenue_growth,
                                        pd.Series(terminal_growth_rate)])
    projected_revneues=pd.concat([projected_revneues,
                                  pd.Series(terminal_revenue)])
    projected_tax_rates = pd.concat([projected_tax_rates,
                                   pd.Series(marginal_tax_rate)])
    projected_reinvestment = pd.concat([projected_reinvestment,
                                        pd.Series(terminal_reinvestment)])
    projected_operating_margins = pd.concat([projected_operating_margins,
                                        pd.Series(terminal_operating_margin)])
    projected_operating_income = pd.concat([projected_operating_income,
                                            pd.Series(terminal_operating_income)])
    projected_operating_income_after_tax = pd.concat([projected_operating_income_after_tax,
                                                      pd.Series(terminal_operating_income_after_tax)])
    projected_FCFF = pd.concat([projected_FCFF,
                                pd.Series(terminal_FCFF)])
    projected_beta = pd.concat([projected_beta,
                                pd.Series(terminal_beta)])
    
    sales_to_capital_ratios = pd.concat([sales_to_capital_ratios,
                                pd.Series([terminal_sales_to_capital_ratio])])
    
    df_valuation = pd.DataFrame({"cumWACC":projected_cost_of_capital_cumulative,
                                 "cumCostOfEquity":projected_cost_of_equity_cumulative,
                                'beta':projected_beta,
                                'revenueGrowth':projected_revenue_growth,
                                "revneues":projected_revneues,
                                 "margins":projected_operating_margins,
                                 'ebit':projected_operating_income,
                                 "sales_to_capital_ratio":sales_to_capital_ratios,
                                "taxRate":projected_tax_rates,
                               'afterTaxOperatingIncome':projected_operating_income_after_tax,
                               "reinvestment":projected_reinvestment,
                               'FCFF':projected_FCFF,
                               'ERP':ERP})
    #### Add reinvestment rate and expected growth rate
    df_valuation['reinvestmentRate'] = df_valuation['reinvestment']/df_valuation['afterTaxOperatingIncome']

    df_valuation['PVFCFF'] = df_valuation['FCFF']/df_valuation['cumWACC']
    value_of_operating_assets = df_valuation['PVFCFF'].sum()
    firm_value =  pd.Series(value_of_operating_assets + cash_and_non_operating_asset)[0]
    equity_value = firm_value - debt_value
    equity_value = pd.Series(equity_value)[0]
    return({'valuation':df_valuation,
            'firm_value':firm_value,
            'equity_value':equity_value,
            'cash_and_non_operating_asset':cash_and_non_operating_asset,
            'debt_value':debt_value,
            'value_of_operating_assets':value_of_operating_assets})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
def valuator_multi_phase(
    risk_free_rate,
    ERP,
    equity_value,
    debt_value,
    unlevered_beta,
            terminal_unlevered_beta,
            year_beta_begins_to_converge_to_terminal_beta,
            current_pretax_cost_of_debt,
            terminal_pretax_cost_of_debt,
            year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
            current_effective_tax_rate,
            marginal_tax_rate,
            year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
             revenue_base,
             revenue_growth_rate_cycle1_begin,
             revenue_growth_rate_cycle1_end,
             revenue_growth_rate_cycle2_begin,
             revenue_growth_rate_cycle2_end,
             revenue_growth_rate_cycle3_begin,
             revenue_growth_rate_cycle3_end,
            revenue_convergance_periods_cycle1,
            revenue_convergance_periods_cycle2,
            revenue_convergance_periods_cycle3,
            length_of_cylcle1,
            length_of_cylcle2,
            length_of_cylcle3,
            current_sales_to_capital_ratio,
            terminal_sales_to_capital_ratio,
            year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital,
            current_operating_margin,
            terminal_operating_margin,
            year_operating_margin_begins_to_converge_to_terminal_operating_margin,
            additional_return_on_cost_of_capital_in_perpetuity=0.0,
            cash_and_non_operating_asset=0.0,
             asset_liquidation_during_negative_growth=0):
    """Value business with 3 cycle in revenue"""
    valuation_interval_in_years = length_of_cylcle1 + length_of_cylcle2 + length_of_cylcle3
    terminal_growth_rate = revenue_growth_rate_cycle3_end
    ### Estimate Cost of Capital during the valution cycle
    projected_cost_of_capital, projected_beta , terminal_beta , projected_cost_of_equity = cost_of_capital_projector(unlevered_beta=unlevered_beta,
                                                          terminal_unlevered_beta=terminal_unlevered_beta,
                                                          current_pretax_cost_of_debt=current_pretax_cost_of_debt,
                                                          terminal_pretax_cost_of_debt=terminal_pretax_cost_of_debt,
                                                          equity_value=equity_value,
                                                          debt_value=debt_value,
                                                          marginal_tax_rate=marginal_tax_rate,
                                                          risk_free_rate=risk_free_rate,
                                                          ERP=ERP,
                                                          valuation_interval_in_years=valuation_interval_in_years,
                                                          year_beta_begins_to_converge_to_terminal_beta=year_beta_begins_to_converge_to_terminal_beta,
                                                          year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt=year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt)
    
    projected_cost_of_capital_cumulative= (1+projected_cost_of_capital).cumprod()
    projected_cost_of_equity_cumulative= (1+projected_cost_of_equity).cumprod()
    ### Estimate Future Revenues
    # projected_revneues,projected_revenue_growth = revenue_projector(revenue_base=revenue_base,
    #                                        revenue_growth_rate=revenue_growth_rate,
    #                                        terminal_growth_rate=terminal_growth_rate,
    #                                        valuation_interval_in_years=valuation_interval_in_years,
    #                                        year_revenue_growth_begin_to_converge_to_terminal_growth_rate= year_revenue_growth_begin_to_converge_to_terminal_growth_rate)
    
    projected_revneues,projected_revenue_growth = revenue_projector_multi_phase(revenue_base = revenue_base,
                                  revenue_growth_rate_cycle1_begin = revenue_growth_rate_cycle1_begin,
                                  revenue_growth_rate_cycle1_end = revenue_growth_rate_cycle1_end,
                                  revenue_growth_rate_cycle2_begin = revenue_growth_rate_cycle2_begin,
                                  revenue_growth_rate_cycle2_end = revenue_growth_rate_cycle2_end,
                                  revenue_growth_rate_cycle3_begin = revenue_growth_rate_cycle3_begin,
                                  revenue_growth_rate_cycle3_end = revenue_growth_rate_cycle3_end,
                                  length_of_cylcle1=length_of_cylcle1,
                                  length_of_cylcle2=length_of_cylcle2,
                                  length_of_cylcle3=length_of_cylcle3,
                                  revenue_convergance_periods_cycle1 = revenue_convergance_periods_cycle1,
                                  revenue_convergance_periods_cycle2 = revenue_convergance_periods_cycle2,
                                  revenue_convergance_periods_cycle3 = revenue_convergance_periods_cycle3)  
    ### Estmimate tax rates
    projected_tax_rates = tax_rate_projector(current_effective_tax_rate=current_effective_tax_rate,
                                            marginal_tax_rate=marginal_tax_rate,
                                            valuation_interval_in_years=valuation_interval_in_years,
                                            year_effective_tax_rate_begin_to_converge_marginal_tax_rate=year_effective_tax_rate_begin_to_converge_marginal_tax_rate)
    ### Estimate slaes to capital ratio during valuation for reinvestment
    sales_to_capital_ratios = sales_to_capital_projector(current_sales_to_capital_ratio,
                               terminal_sales_to_capital_ratio,
                               valuation_interval_in_years=valuation_interval_in_years,
                               year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital=year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital)

    ### Estimate Reinvestemnt
    projected_reinvestment = reinvestment_projector(revenue_base=revenue_base,
                                                    projected_revneues = projected_revneues,
                                                    sales_to_capital_ratios=sales_to_capital_ratios,
                                                    asset_liquidation_during_negative_growth=asset_liquidation_during_negative_growth)
    ### Operating Margin
    projected_operating_margins = operating_margin_projector(current_operating_margin,
                                                            terminal_operating_margin,
                                                            valuation_interval_in_years=valuation_interval_in_years,
                                                            year_operating_margin_begins_to_converge_to_terminal_operating_margin=year_operating_margin_begins_to_converge_to_terminal_operating_margin)
    ###EBIT
    projected_operating_income = projected_revneues * projected_operating_margins
    ### After Tax EBIT (EBI)
    projected_operating_income_after_tax = (projected_operating_income*(1-projected_tax_rates))
    ### FCFF: EBI-Reinvestment
    projected_FCFF = projected_operating_income_after_tax - projected_reinvestment
    
    ### Compute Terminal Value
    terminal_cost_of_capital = projected_cost_of_capital[-1:].values
    terminal_cost_of_equity = projected_cost_of_equity[-1:].values
    if terminal_growth_rate < 0:
        terminal_reinvestment_rate=0
    else:
        terminal_reinvestment_rate = terminal_growth_rate/(terminal_cost_of_capital+additional_return_on_cost_of_capital_in_perpetuity)
    terminal_revenue = projected_revneues[-1:].values * (1+terminal_growth_rate)
    terminal_operating_income = terminal_revenue * terminal_operating_margin
    terminal_operating_income_after_tax = terminal_operating_income*(1-marginal_tax_rate)
    terminal_reinvestment = terminal_operating_income_after_tax* terminal_reinvestment_rate
    terminal_FCFF = terminal_operating_income_after_tax - terminal_reinvestment
    termimal_discount_rate = (terminal_cost_of_capital-terminal_growth_rate)*(1+projected_cost_of_capital).prod()
    termimal_equity_discount_rate = (terminal_cost_of_equity-terminal_growth_rate)*(1+projected_cost_of_equity).prod()
    
    ### Concatinate Projected Values with termianl values
    projected_cost_of_capital_cumulative=pd.concat([projected_cost_of_capital_cumulative,
                                                    pd.Series(termimal_discount_rate)])
    projected_cost_of_equity_cumulative = pd.concat([projected_cost_of_equity_cumulative,
                                                    pd.Series(termimal_equity_discount_rate)])

    projected_revenue_growth = pd.concat([projected_revenue_growth,
                                        pd.Series(terminal_growth_rate)])
    projected_revneues=pd.concat([projected_revneues,
                                  pd.Series(terminal_revenue)])
    projected_tax_rates = pd.concat([projected_tax_rates,
                                   pd.Series(marginal_tax_rate)])
    projected_reinvestment = pd.concat([projected_reinvestment,
                                        pd.Series(terminal_reinvestment)])
    projected_operating_margins = pd.concat([projected_operating_margins,
                                        pd.Series(terminal_operating_margin)])
    projected_operating_income = pd.concat([projected_operating_income,
                                            pd.Series(terminal_operating_income)])
    projected_operating_income_after_tax = pd.concat([projected_operating_income_after_tax,
                                                      pd.Series(terminal_operating_income_after_tax)])
    projected_FCFF = pd.concat([projected_FCFF,
                                pd.Series(terminal_FCFF)])
    projected_beta = pd.concat([projected_beta,
                                pd.Series(terminal_beta)])
    
    sales_to_capital_ratios = pd.concat([sales_to_capital_ratios,
                                pd.Series([terminal_sales_to_capital_ratio])])
    
    df_valuation = pd.DataFrame({"cumWACC":projected_cost_of_capital_cumulative,
                                 "cumCostOfEquity":projected_cost_of_equity_cumulative,
                                'beta':projected_beta,
                                'revenueGrowth':projected_revenue_growth,
                                "revneues":projected_revneues,
                                 "margins":projected_operating_margins,
                                 'ebit':projected_operating_income,
                                 "sales_to_capital_ratio":sales_to_capital_ratios,
                                "taxRate":projected_tax_rates,
                               'afterTaxOperatingIncome':projected_operating_income_after_tax,
                               "reinvestment":projected_reinvestment,
                               'FCFF':projected_FCFF,
                               'ERP':ERP})
    #### Add reinvestment rate and expected growth rate
    df_valuation['reinvestmentRate'] = df_valuation['reinvestment']/df_valuation['afterTaxOperatingIncome']

    df_valuation['PVFCFF'] = df_valuation['FCFF']/df_valuation['cumWACC']
    value_of_operating_assets = df_valuation['PVFCFF'].sum()
    firm_value =  pd.Series(value_of_operating_assets + cash_and_non_operating_asset)[0]
    equity_value = firm_value - debt_value
    equity_value = pd.Series(equity_value)[0]
    return({'valuation':df_valuation,
            'firm_value':firm_value,
            'equity_value':equity_value,
            'cash_and_non_operating_asset':cash_and_non_operating_asset,
            'debt_value':debt_value,
            'value_of_operating_assets':value_of_operating_assets})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Monte Carlo Simulation

In [85]:
def monte_carlo_valuator(unlevered_beta,
                        terminal_unlevered_beta,
                         current_pretax_cost_of_debt,
                         terminal_pretax_cost_of_debt,
                         risk_free_rate,
                         ERP,
                         revenue_base,
                         revenue_growth_rate,
                         terminal_growth_rate,
                         current_effective_tax_rate,
                         marginal_tax_rate,
                          current_sales_to_capital_ratio,
                          terminal_sales_to_capital_ratio,
                         current_operating_margin,
                         terminal_operating_margin,
                         additional_return_on_cost_of_capital_in_perpetuity,
                         year_revenue_growth_begin_to_converge_to_terminal_growth_rate,
                         year_operating_margin_begins_to_converge_to_terminal_operating_margin,
                         year_beta_begins_to_converge_to_terminal_beta,
                         year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
                         year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
                          year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital,
                         equity_value,
                         debt_value,
                         cash_and_non_operating_asset,
                         list_of_correlation_between_variables=[['revenue_growth_rate','terminal_operating_margin',0.4],
                                                                ['additional_return_on_cost_of_capital_in_perpetuity','terminal_operating_margin',.6]],
                         asset_liquidation_during_negative_growth=0,
                         sample_size=1000,
                         valuation_interval_in_years=10):
    """
    ### Example input data
    unlevered_beta=ot.Triangular(1.15,1.2,1.25),
    terminal_unlevered_beta = ot.Triangular(1.0,1.05,1.1),
    current_pretax_cost_of_debt = ot.Triangular(0.02,.025,0.03),
    terminal_pretax_cost_of_debt = ot.Triangular(0.024,.027,0.029),
    risk_free_rate = ot.Normal(0.025,.001),
    ERP = ot.Normal(0.0575,.005),
    revenue_base = ot.Triangular(257637000000*.98,257637000000,257637000000*1.02),
    revenue_growth_rate = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm(4, loc=.05, scale=0.03))),
    current_effective_tax_rate = ot.Triangular(.15,.16,.17),
    marginal_tax_rate = ot.Triangular(.20,.21,.22),
    sales_to_capital_ratio = ot.Triangular(1.5,1.8,2.3),
    current_operating_margin = ot.Triangular(0.30,.32,.33),
    terminal_operating_margin = ot.Triangular(0.25,.27,.29),
    additional_return_on_cost_of_capital_in_perpetuity= ot.Triangular(0.035,0.045,0.055),
    equity_value = ot.Triangular(1888290078720*.95,1888290078720,1888290078720*1.05),
    debt_value = ot.Triangular(28395000000*.95,28395000000,28395000000*1.05),
    cash_and_non_operating_asset= ot.Uniform(164188850000,164188850001),
    year_revenue_growth_begin_to_converge_to_risk_free = ot.Uniform(5,6),
    year_operating_margin_begins_to_converge_to_terminal_operating_margin = ot.Uniform(5,6),
    year_beta_begins_to_converge_to_terminal_beta=ot.Uniform(5,6),
    year_effective_tax_rate_begin_to_converge_marginal_tax_rate = ot.Uniform(5,6),
    year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt = ot.Uniform(5,6),
    list_of_correlation_between_variables=[['revenue_growth_rate','terminal_operating_margin',0.4],
                                           ['additional_return_on_cost_of_capital_in_perpetuity','terminal_operating_margin',.6]],
    sample_size=3000,
    valuation_interval_in_years=10)
    """
    variables_distributsion = [unlevered_beta,
                               terminal_unlevered_beta,
                               current_pretax_cost_of_debt,
                               terminal_pretax_cost_of_debt,
                               risk_free_rate,
                               ERP,
                               revenue_base,
                               revenue_growth_rate,
                               terminal_growth_rate,
                               current_effective_tax_rate,
                               marginal_tax_rate,
                               current_sales_to_capital_ratio,
                               terminal_sales_to_capital_ratio,
                               current_operating_margin,
                               terminal_operating_margin,
                               additional_return_on_cost_of_capital_in_perpetuity,
                               equity_value,
                               debt_value,
                               cash_and_non_operating_asset,
                               year_revenue_growth_begin_to_converge_to_terminal_growth_rate,
                               year_operating_margin_begins_to_converge_to_terminal_operating_margin,
                               year_beta_begins_to_converge_to_terminal_beta,
                               year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
                               year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
                               year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital]
    variable_names = ['unlevered_beta',
                      'terminal_unlevered_beta',
                      'current_pretax_cost_of_debt',
                    'terminal_pretax_cost_of_debt',
                    'risk_free_rate',
                    'ERP',
                    'revenue_base',
                    'revenue_growth_rate',
                    'terminal_growth_rate',
                      'current_effective_tax_rate',
                      'marginal_tax_rate',
                      'current_sales_to_capital_ratio',
                      'terminal_sales_to_capital_ratio',
                      'current_operating_margin',
                      'terminal_operating_margin',
                      'additional_return_on_cost_of_capital_in_perpetuity',
                      'equity_value',
                      'debt_value',
                      'cash_and_non_operating_asset',
                      'year_revenue_growth_begin_to_converge_to_terminal_growth_rate',
                      'year_operating_margin_begins_to_converge_to_terminal_operating_margin',
                      'year_beta_begins_to_converge_to_terminal_beta',
                      'year_effective_tax_rate_begin_to_converge_marginal_tax_rate',
                      'year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt',
                      'year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital']
    list_of_columns_with_year_to_be_int = [s for s in variable_names if "year" in s]
    ### Build a DataFarame to have index - location of each variable in the correlation matrix
    dict_of_varible = dict(zip(variable_names,
                            range(0,len(variable_names))))
    df_variables = pd.DataFrame([dict_of_varible])

    ### Initaile Correlation Matrix
    R = ot.CorrelationMatrix(len(variables_distributsion))
    ### pair correlation between each variable
    for pair_of_variable in list_of_correlation_between_variables:
        location = df_variables[pair_of_variable[:2]].values[0]
        #print(location)
        R[int(location[0]),int(location[1])] = pair_of_variable[2]

    ### Build the correlation into composed distribution function
    ### For ot.NormalCopula The correlation matrix must be definite positive 
    ### Here is an implementaion on how to get the nearest psd matirx https://stackoverflow.com/questions/43238173/python-convert-matrix-to-positive-semi-definite
    copula = ot.NormalCopula(R)
    BuiltComposedDistribution = ot.ComposedDistribution(variables_distributsion,
                                                        copula)
    ### Generate samples
    generated_sample = BuiltComposedDistribution.getSample(sample_size)
    df_generated_sample = pd.DataFrame.from_records(generated_sample, columns= variable_names)
    df_generated_sample[list_of_columns_with_year_to_be_int] = df_generated_sample[list_of_columns_with_year_to_be_int].apply(lambda x: round(x))
    print("Scenario Generation Complete")
    ### 
    df_generated_sample['full_valuation']= df_generated_sample.apply(lambda row:
                                                                    valuator(
                                                                        unlevered_beta= row['unlevered_beta'] ,
                                                                        terminal_unlevered_beta= row['terminal_unlevered_beta'],
                                                                        current_pretax_cost_of_debt= row['current_pretax_cost_of_debt'],
                                                                        terminal_pretax_cost_of_debt= row['terminal_pretax_cost_of_debt'],
                                                                        equity_value= row['equity_value'],
                                                                        debt_value= row['debt_value'],
                                                                        marginal_tax_rate=row['marginal_tax_rate'],
                                                                        risk_free_rate=row['risk_free_rate'],
                                                                        ERP=row['ERP'],
                                                                        valuation_interval_in_years=valuation_interval_in_years,
                                                                        year_beta_begins_to_converge_to_terminal_beta = row['year_beta_begins_to_converge_to_terminal_beta'],
                                                                        year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt= row['year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt'],
                                                                        revenue_growth_rate= row['revenue_growth_rate'],
                                                                        terminal_growth_rate = row['terminal_growth_rate'],
                                                                        revenue_base= row['revenue_base'],
                                                                        year_revenue_growth_begin_to_converge_to_terminal_growth_rate= row['year_revenue_growth_begin_to_converge_to_terminal_growth_rate'],
                                                                        current_effective_tax_rate=row['current_effective_tax_rate'],
                                                                        year_effective_tax_rate_begin_to_converge_marginal_tax_rate= row['year_effective_tax_rate_begin_to_converge_marginal_tax_rate'],
                                                                        current_sales_to_capital_ratio= row['current_sales_to_capital_ratio'],
                                                                        terminal_sales_to_capital_ratio = row['terminal_sales_to_capital_ratio'],
                                                                        year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital= row['year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital'],
                                                                        current_operating_margin=row['current_operating_margin'],
                                                                        terminal_operating_margin=row['terminal_operating_margin'],
                                                                        year_operating_margin_begins_to_converge_to_terminal_operating_margin= row['year_operating_margin_begins_to_converge_to_terminal_operating_margin'],
                                                                        additional_return_on_cost_of_capital_in_perpetuity= row['additional_return_on_cost_of_capital_in_perpetuity'],
                                                                        cash_and_non_operating_asset =  row['cash_and_non_operating_asset'],
                                                                        asset_liquidation_during_negative_growth=asset_liquidation_during_negative_growth),
                                                                        axis=1)
    ### extract the valuation result
    df_generated_sample['valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['valuation'])
    df_generated_sample['equity_valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['equity_value'])
    df_generated_sample['firm_valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['firm_value'])
    df_generated_sample['terminal_revenue'] = df_generated_sample['valuation'].apply(lambda x: x['revneues'].values[-1])
    df_generated_sample['terminal_operating_margin'] = df_generated_sample['valuation'].apply(lambda x: x['margins'].values[-1])
    df_generated_sample['terminal_reinvestmentRate'] = df_generated_sample['valuation'].apply(lambda x: x['reinvestmentRate'].values[-1])
    df_generated_sample['terminal_afterTaxOperatingIncome'] = df_generated_sample['valuation'].apply(lambda x: x['afterTaxOperatingIncome'].values[-1])
    df_generated_sample['terminal_FCFF'] = df_generated_sample['valuation'].apply(lambda x: x['FCFF'].values[-1])
    return(df_generated_sample)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [86]:
def monte_carlo_valuator_multi_phase(
    risk_free_rate,
    ERP,
    equity_value,
    debt_value,
    unlevered_beta,
    terminal_unlevered_beta,
    year_beta_begins_to_converge_to_terminal_beta,
    current_pretax_cost_of_debt,
    terminal_pretax_cost_of_debt,
    year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
    current_effective_tax_rate,
    marginal_tax_rate,
    year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
     revenue_base,
     revenue_growth_rate_cycle1_begin,
     revenue_growth_rate_cycle1_end,
     revenue_growth_rate_cycle2_begin,
     revenue_growth_rate_cycle2_end,
     revenue_growth_rate_cycle3_begin,
     revenue_growth_rate_cycle3_end,
    revenue_convergance_periods_cycle1,
    revenue_convergance_periods_cycle2,
    revenue_convergance_periods_cycle3,
    length_of_cylcle1,
    length_of_cylcle2,
    length_of_cylcle3,
    current_sales_to_capital_ratio,
    terminal_sales_to_capital_ratio,
    year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital,
    current_operating_margin,
    terminal_operating_margin,
    year_operating_margin_begins_to_converge_to_terminal_operating_margin,
    additional_return_on_cost_of_capital_in_perpetuity,
    cash_and_non_operating_asset,
    asset_liquidation_during_negative_growth,
    sample_size=1000,
    list_of_correlation_between_variables=[['additional_return_on_cost_of_capital_in_perpetuity','terminal_sales_to_capital_ratio',0.4],
                                           ['additional_return_on_cost_of_capital_in_perpetuity','terminal_operating_margin',.6]]):
    variables_distributsion = [risk_free_rate,
                                   ERP,
                                   equity_value,
                                   debt_value,
                                   unlevered_beta,
                                    terminal_unlevered_beta,
                                    year_beta_begins_to_converge_to_terminal_beta,
                                    current_pretax_cost_of_debt,
                                    terminal_pretax_cost_of_debt,
                                    year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt,
                                    current_effective_tax_rate,
                                    marginal_tax_rate,
                                    year_effective_tax_rate_begin_to_converge_marginal_tax_rate,
                                    revenue_base,
                                    revenue_growth_rate_cycle1_begin,
                                    revenue_growth_rate_cycle1_end,
                                    revenue_growth_rate_cycle2_begin,
                                    revenue_growth_rate_cycle2_end,
                                    revenue_growth_rate_cycle3_begin,
                                    revenue_growth_rate_cycle3_end,
                                    revenue_convergance_periods_cycle1,
                                    revenue_convergance_periods_cycle2,
                                    revenue_convergance_periods_cycle3,
                                    length_of_cylcle1,
                                    length_of_cylcle2,
                                    length_of_cylcle3,
                                    current_sales_to_capital_ratio,
                                    terminal_sales_to_capital_ratio,
                                    year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital,
                                    current_operating_margin,
                                    terminal_operating_margin,
                                    year_operating_margin_begins_to_converge_to_terminal_operating_margin,
                                    additional_return_on_cost_of_capital_in_perpetuity,
                                    cash_and_non_operating_asset,
                                    asset_liquidation_during_negative_growth]
    variable_names = ['risk_free_rate',
                                   'ERP',
                                   'equity_value',
                                   'debt_value',
                                   'unlevered_beta',
                                    'terminal_unlevered_beta',
                                    'year_beta_begins_to_converge_to_terminal_beta',
                                    'current_pretax_cost_of_debt',
                                    'terminal_pretax_cost_of_debt',
                                    'year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt',
                                    'current_effective_tax_rate',
                                    'marginal_tax_rate',
                                    'year_effective_tax_rate_begin_to_converge_marginal_tax_rate',
                                    'revenue_base',
                                    'revenue_growth_rate_cycle1_begin',
                                    'revenue_growth_rate_cycle1_end',
                                    'revenue_growth_rate_cycle2_begin',
                                    'revenue_growth_rate_cycle2_end',
                                    'revenue_growth_rate_cycle3_begin',
                                    'revenue_growth_rate_cycle3_end',
                                    'revenue_convergance_periods_cycle1',
                                    'revenue_convergance_periods_cycle2',
                                    'revenue_convergance_periods_cycle3',
                                    'length_of_cylcle1',
                                    'length_of_cylcle2',
                                    'length_of_cylcle3',
                                    'current_sales_to_capital_ratio',
                                    'terminal_sales_to_capital_ratio',
                                    'year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital',
                                    'current_operating_margin',
                                    'terminal_operating_margin',
                                    'year_operating_margin_begins_to_converge_to_terminal_operating_margin',
                                    'additional_return_on_cost_of_capital_in_perpetuity',
                                    'cash_and_non_operating_asset',
                                    'asset_liquidation_during_negative_growth']
    ### The following variable  should have "year" in their definition but I did not think of ut. So I am adding them to list_of_columns_with_year_to_be_int 
    list_of_columns_with_year_to_be_int = [s for s in variable_names if "year" in s] +['length_of_cylcle1','length_of_cylcle2','length_of_cylcle3',
                                                                                       'revenue_convergance_periods_cycle1',
                                                                                       'revenue_convergance_periods_cycle2','revenue_convergance_periods_cycle3']
    ### Build a DataFarame to have index - location of each variable in the correlation matrix
    dict_of_varible = dict(zip(variable_names,
                            range(0,len(variable_names))))
    df_variables = pd.DataFrame([dict_of_varible])

    ### Initaile Correlation Matrix
    R = ot.CorrelationMatrix(len(variables_distributsion))
    ### pair correlation between each variable
    for pair_of_variable in list_of_correlation_between_variables:
        location = df_variables[pair_of_variable[:2]].values[0]
        #print(location)
        R[int(location[0]),int(location[1])] = pair_of_variable[2]

    ### Build the correlation into composed distribution function
    ### For ot.NormalCopula The correlation matrix must be definite positive 
    ### Here is an implementaion on how to get the nearest psd matirx https://stackoverflow.com/questions/43238173/python-convert-matrix-to-positive-semi-definite
    copula = ot.NormalCopula(R)
    BuiltComposedDistribution = ot.ComposedDistribution(variables_distributsion,
                                                        copula)
    ### Generate samples
    generated_sample = BuiltComposedDistribution.getSample(sample_size)
    df_generated_sample = pd.DataFrame.from_records(generated_sample, columns= variable_names)
    df_generated_sample[list_of_columns_with_year_to_be_int] = df_generated_sample[list_of_columns_with_year_to_be_int].apply(lambda x: round(x))
    print("Scenario Generation Complete")
    ###
    df_generated_sample['full_valuation']= df_generated_sample.apply(lambda row:
                                                                    valuator_multi_phase(
                                                                        risk_free_rate = row['risk_free_rate'],
                                                                        ERP = row['ERP'],
                                                                        equity_value = row['equity_value'],
                                                                        debt_value = row['debt_value'],
                                                                        unlevered_beta = row['unlevered_beta'],
                                                                        terminal_unlevered_beta = row['terminal_unlevered_beta'],
                                                                        year_beta_begins_to_converge_to_terminal_beta = row['year_beta_begins_to_converge_to_terminal_beta'],
                                                                        current_pretax_cost_of_debt = row['current_pretax_cost_of_debt'],
                                                                        terminal_pretax_cost_of_debt = row['terminal_pretax_cost_of_debt'],
                                                                        year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt = row['year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt'],
                                                                        current_effective_tax_rate = row['current_effective_tax_rate'],
                                                                        marginal_tax_rate = row['marginal_tax_rate'],
                                                                        year_effective_tax_rate_begin_to_converge_marginal_tax_rate = row['year_effective_tax_rate_begin_to_converge_marginal_tax_rate'],
                                                                        revenue_base = row['revenue_base'],
                                                                        revenue_growth_rate_cycle1_begin = row['revenue_growth_rate_cycle1_begin'],
                                                                        revenue_growth_rate_cycle1_end = row['revenue_growth_rate_cycle1_end'],
                                                                        revenue_growth_rate_cycle2_begin = row['revenue_growth_rate_cycle2_begin'],
                                                                        revenue_growth_rate_cycle2_end = row['revenue_growth_rate_cycle2_end'],
                                                                        revenue_growth_rate_cycle3_begin = row['revenue_growth_rate_cycle3_begin'],
                                                                        revenue_growth_rate_cycle3_end = row['revenue_growth_rate_cycle3_end'],
                                                                        revenue_convergance_periods_cycle1 = row['revenue_convergance_periods_cycle1'],
                                                                        revenue_convergance_periods_cycle2 = row['revenue_convergance_periods_cycle2'],
                                                                        revenue_convergance_periods_cycle3 = row['revenue_convergance_periods_cycle3'],
                                                                        length_of_cylcle1 = row['length_of_cylcle1'],
                                                                        length_of_cylcle2 = row['length_of_cylcle2'],
                                                                        length_of_cylcle3 = row['length_of_cylcle3'],
                                                                        current_sales_to_capital_ratio = row['current_sales_to_capital_ratio'],
                                                                        terminal_sales_to_capital_ratio = row['terminal_sales_to_capital_ratio'],
                                                                        year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital = row['year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital'],
                                                                        current_operating_margin = row['current_operating_margin'],
                                                                        terminal_operating_margin = row['terminal_operating_margin'],
                                                                        year_operating_margin_begins_to_converge_to_terminal_operating_margin = row['year_operating_margin_begins_to_converge_to_terminal_operating_margin'],
                                                                        additional_return_on_cost_of_capital_in_perpetuity = row['additional_return_on_cost_of_capital_in_perpetuity'],
                                                                        cash_and_non_operating_asset = row['cash_and_non_operating_asset'],
                                                                        asset_liquidation_during_negative_growth=row['asset_liquidation_during_negative_growth']),
                                                                        axis=1)
    ### extract the valuation result
    df_generated_sample['valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['valuation'])
    df_generated_sample['equity_valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['equity_value'])
    df_generated_sample['firm_valuation'] = df_generated_sample['full_valuation'].apply(lambda x: x['firm_value'])
    df_generated_sample['terminal_revenue'] = df_generated_sample['valuation'].apply(lambda x: x['revneues'].values[-1])
    df_generated_sample['terminal_operating_margin'] = df_generated_sample['valuation'].apply(lambda x: x['margins'].values[-1])
    df_generated_sample['terminal_reinvestmentRate'] = df_generated_sample['valuation'].apply(lambda x: x['reinvestmentRate'].values[-1])
    df_generated_sample['terminal_afterTaxOperatingIncome'] = df_generated_sample['valuation'].apply(lambda x: x['afterTaxOperatingIncome'].values[-1])
    df_generated_sample['terminal_FCFF'] = df_generated_sample['valuation'].apply(lambda x: x['FCFF'].values[-1])
    return(df_generated_sample)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Plotly Charts

In [87]:
def histogram_plotter_plotly(data,
                              colmn_name,
                              xlabel,
                              title='Data',
                              bins=30,
                              percentile=[15,50,85],
                              color=['green','yellow','red'],
                              histnorm='percent',
                              marginal=None,
                              height=470,
                              width=670):
    """Plot Historgam via Plotly"""
    fig = px.histogram(data,
                       x=colmn_name,
                       histnorm=histnorm,
                       nbins=bins,
                       labels={colmn_name:xlabel},
                       marginal=marginal)
    ### Make an educated guess on the y_max for line on the historgram
    n, bin_edges = np.histogram(data[colmn_name],bins=bins,density=False)
    bin_probability = n/float(n.sum())
    y_max = np.max(n/(n.sum())*100) *1.65
    ### Ad trace of percentiles
    for i in range(len(percentile)):
        fig = fig.add_trace(go.Scatter(x=[np.percentile(data[colmn_name],percentile[i]), np.percentile(data[colmn_name],percentile[i])],
                                       y=(0,y_max), 
                                       mode="lines",
                                       name= str(percentile[i])+' Percentile',
                                       marker=dict(color=color[i])))
        #fig = fig.add_vline(x = np.percentile(data[colmn_name],percentile[i]), line_dash = 'dash',line_color=color[i])
        #print(str(percentile[i])+" Percentile",np.percentile(data[colmn_name],percentile[i]))
        fig.update_layout(height=height, width=width,title=title,
                          legend=dict(orientation="v"))
    return(fig)

def ecdf_plotter_plotly(data,
                              colmn_name,
                              xlabel,
                              title='Data',
                              percentile=[15,50,85],
                              color=['green','yellow','red'],
                              marginal=None,
                              height=500,
                              width=700):
    """Plot ECDF via Plotly"""
    fig = px.ecdf(data,
                     x=colmn_name,
                     labels={colmn_name:xlabel},
                     marginal=marginal)
    for i in range(len(percentile)):
        fig = fig.add_trace(go.Scatter(x=[np.percentile(data[colmn_name],percentile[i]), np.percentile(data[colmn_name],percentile[i])],
                                       y=(0,1), 
                                       mode="lines",
                                       name= str(percentile[i])+' Percentile',
                                       marker=dict(color=color[i])))
        #fig = fig.add_vline(x = np.percentile(data[colmn_name],percentile[i]), line_dash = 'dash',line_color=color[i])
        #print(str(percentile[i])+" Percentile",np.percentile(data[colmn_name],percentile[i]))
        fig.update_layout(height=height, width=width,title=title,
                          legend=dict(orientation="v"))
    return(fig)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [88]:
def time_series_plotly(df_data,
                       x,
                       yleft,
                       yright,
                       height=500, 
                       width=1600,
                       title=None):
    """ Graph 2 time series on 2 different y-axis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df_data[x], y=df_data[yleft], name=yleft),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df_data[x], y=df_data[yright], name=yright),
        secondary_y=True,
    )
    fig = fig.update_layout(height=height, width=width,title=title)
    return(fig)

def plotly_line_bar_chart(df_data,
                       x,
                       ybar,
                       yline,
                       height=500, 
                       width=1600,
                       rangemode=None,
                       title=None):
    """ Graph 2 time series on 2 different y-axis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    #fig.update_yaxes(rangemode='tozero')

    for bar_var in ybar:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x], y=df_data[bar_var],name=bar_var),
            secondary_y=False
            )
    #fig.update_yaxes(rangemode='tozero')
    for line_var in yline:
        fig.add_trace(
            go.Scatter(x=df_data[x], y=df_data[line_var],name=line_var),
            secondary_y=True,
            )
    if rangemode != None:
        fig.update_yaxes(rangemode=rangemode)
    fig = fig.update_layout(height=height, width=width,title=title)
    return(fig)


def plotly_line_dash_bar_chart(df_data,
                       x,
                       ybar,
                       yline,
                       ydash,
                       height=500, 
                       width=1600,
                       rangemode=None,
                       title=None,
                       barmode='group',
                       texttemplate= "%{value}"
                       ):
    """ Graph 2 time series on 2 different y-axis"""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    #fig.update_yaxes(rangemode='tozero')

    for bar_var in ybar:
        # Add traces
        fig.add_trace(
            go.Bar(x=df_data[x],
                   y=df_data[bar_var],
                   name=bar_var,
                   text = df_data[bar_var],
                   textposition="inside",
                   texttemplate= texttemplate,
                   textfont_color="white"),
            secondary_y=False,
            )
    for line_var in yline:
        fig.add_trace(
            go.Scatter(x=df_data[x], 
                       y=df_data[line_var],
                       name=line_var
                       ),
            secondary_y=True,
            )
        
    for dash_var in ydash:
        fig.add_trace(
            go.Scatter(x=df_data[x],
                       y=df_data[dash_var],
                       name=dash_var,
                       line = dict(dash='dot')),
            secondary_y=True,
            )
    if rangemode != None:
        fig.update_yaxes(rangemode=rangemode)
    fig = fig.update_layout(height=height, 
                            width=width,
                            title=title,
                            barmode=barmode)
    return(fig)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Company Analyzer

In [89]:
def financial_data_retuner(df_prices_fin,
                     target_symbol,
                     date=dt.datetime(2002,1,1)):
    df_prices_fin = df_prices_fin[df_prices_fin['Date']>date].copy()
    df_finacials = df_prices_fin.drop_duplicates(['symbol','dateFinncialEnding'],keep='last').copy()
    df_target_symbol_fin = df_finacials[df_finacials['symbol']==target_symbol].copy()
    df_target_symbol_fin['dateFinncialEndingHover'] = df_target_symbol_fin['dateFinncialEnding'].astype(str).apply(lambda x: x[:10])
    return(df_target_symbol_fin)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [90]:
def company_analyzer(df_prices_fin,
                     target_symbol,
                     date=dt.datetime(2002,1,1)):
    """If running to error try dropping fiancialEndingYear null records"""
    list_of_chart=[]
    try:
        s = fmpsdk.company_profile(my_fmp_api,target_symbol)[0]
        s1 = s['description'].split()
        s1 = np.array(s1)
        s2 = np.array_split(s1,len(s1)/16)
        for i in range(len(s2)):
            print(' '.join(s2[i]))      
        print("Full time Employeee",s['fullTimeEmployees'])
        print("Website ",s['website']) 
        print("Exchange ",s['exchange'])
        print('CEO',s['ceo'])
    except:
        pass
    ### Stock Adj Price Over Time
    
    fig_stock_adj_price = plotly_line_dash_bar_chart(df_prices_fin[(df_prices_fin['symbol']==target_symbol) & 
                                              (df_prices_fin['Date']>=date)],
                                      x='Date',
                                      yline=['Adj Close'],
                                      ybar=[],  
                                      ydash=[], 
                                      height=500, width=1600, rangemode=None, title='Stock Adj Price', texttemplate='%{value}')
    list_of_chart.append(fig_stock_adj_price)
    ### Filter on Financial Statments
    df_prices_fin = df_prices_fin[df_prices_fin['Date']>date].copy()
    df_finacials = df_prices_fin.drop_duplicates(['symbol','dateFinncialEnding'],keep='last').copy()
    df_target_symbol_fin = df_finacials[df_finacials['symbol']==target_symbol].copy()
    
    df_target_symbol_fin['dateFinncialEndingHover'] = df_target_symbol_fin['dateFinncialEnding'].astype(str).apply(lambda x: x[:10])
    print(target_symbol)
    print(df_prices_fin[df_prices_fin['industry'].isin(set(df_target_symbol_fin['industry']))][['symbol','industry']].drop_duplicates())

    ### Income Statement
    fig_income_statement = plotly_line_dash_bar_chart(df_target_symbol_fin,
                                                       x='dateFinncialEnding',
                                                       ybar=['revenue', 'costOfRevenue',
                              'grossProfit', 'grossProfitRatio', 'researchAndDevelopmentExpenses',
       'generalAndAdministrativeExpenses', 'sellingAndMarketingExpenses',
       'sellingGeneralAndAdministrativeExpenses', 'otherExpenses',
       'operatingExpenses', 'costAndExpenses', 'interestIncome',
       'interestExpense', 'depreciationAndAmortization', 'ebitda',
       'ebitdaratio', 'operatingIncome', 'operatingIncomeRatio',
       'totalOtherIncomeExpensesNet', 'incomeBeforeTax',
       'incomeBeforeTaxRatio', 'incomeTaxExpense', 'netIncome',
       'netIncomeRatio', 'eps', 'epsdiluted', 'weightedAverageShsOut',
       'weightedAverageShsOutDil', 'rndUnamortized', 'rndAmortization',
       'operatingIncomeAdjusted', 'operatingExpensesAdjusted', 'ebit',
       'ebitAdjusted'],
                                yline=[],
                                ydash=[],
                                title='Income Statement',
                                width=1600, height=450)
    list_of_chart.append(fig_income_statement)
    
    ### Beta over time
    fig_beta = px.line(df_prices_fin[df_prices_fin['symbol']==target_symbol].groupby(['symbol','Date'])[
                                                                                            ['companyBetaSample',
                                                                                             'companyBetaIndustry',
                                                                                            'unleveredBetaSampleMedian',
                                                                                            'unleveredBetaIndustryMedian',
                                                                                            'unleveredBetaSectorMedian']].mean().reset_index(),
            x='Date',
            y=['companyBetaSample',
               'companyBetaIndustry',
               'unleveredBetaIndustryMedian',
               'unleveredBetaSampleMedian',
               'unleveredBetaSectorMedian'],
            width=1600, height=450,
            title='Beta')
    list_of_chart.append(fig_beta)

    ### Current and long term Ratio
    fig_curret_ratio = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['totalEquityExcGoodwill','totalStockholdersEquity','totalCurrentAssets','totalCurrentLiabilities','totalAssets','totalLiabilities','goodwill'],
                                yline=['currentRatio','longTermRatio','debtToAsset'],
                                ydash=['currentRatioIndustryMedian','longTermRatioIndustryMedian','debtToAssetIndustryMedian'],
                                title='Current-Long Term Ratio',
                                width=1600, height=450)
    list_of_chart.append(fig_curret_ratio)

    ## cost of Debt
    fig_cost_of_debt = plotly_line_dash_bar_chart(df_target_symbol_fin,
                                                   x='dateFinncialEnding',
                                                   ybar=['debtEquityRatio'],
                                                   yline=['costOfDebtBackwardLooking'],
                                                   ydash=['10YearRF','DAAA','DBAA'],
                                                   title='Cost of Debt',
                                                   width=1550, height=500)
    list_of_chart.append(fig_cost_of_debt)

    ### Debt
    fig_debt = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['totalLiabilitiesAndStockholdersEquity','totalDebt','totalCash'],
                                yline=['debtEquityRatio','debtToAsset','interestExpenseCoverageRatio'],
                                ydash=['debtEquityRatioIndustryMedian','debtToAssetIndustryMedian'],
                                title='Debt',
                                width=1550, height=500)
    list_of_chart.append(fig_debt)

    ### Buffet
    fig_buffet= plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=[],
                                yline=['BuffetSG&ARatio','BuffetresearchAndDevelopmentExpensesRatio',
                                       'BuffetInterestExpenseRatio','BuffetCapExRatio','BuffetCapExAdjustedRatio',
                                       'BuffetDepreciationAndAmortizationRatio','netIncomeRatio'],
                                ydash=['BuffetSG&ARatioIndustryMedian',
                                       'BuffetresearchAndDevelopmentExpensesRatioIndustryMedian',
                                       'BuffetInterestExpenseRatioIndustryMedian',
                                       'BuffetCapExRatioIndustryMedian',
                                    'BuffetCapExAdjustedRatioIndustryMedian',
                                    'BuffetDepreciationAndAmortizationRatioIndustryMedian'],
                                title='Buffet',
                                width=1550, height=500)
    list_of_chart.append(fig_buffet)

    ### Operating Leverage
    fig_operating_leverage = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['revenueChange','revenueYoY',
                              'operatingIncomeChange','operatingIncomeYoY',
                              'operatingIncomeAdjustedChange','operatingIncomeAdjustedYoY'],
                                yline=['operatingLeverage','operatingLeverageYoY',
                                       'operatingLeverageAdjusted','operatingLeverageAdjustedYoY'],
                                ydash=['operatingLeverageIndustryMedian',
                                    'operatingLeverageYoYIndustryMedian',
                                    ],
                                title='Operating Leverage',
                                width=1550, height=500)
    list_of_chart.append(fig_operating_leverage)


    ### Profit revenueTT
    fig_profit = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['revenue',
                              'revenueTTM',
                                'operatingIncome',
                              'operatingIncomeTTM',
                                'operatingIncomeAdjusted',
                              'operatingIncomeAdjustedTTM',
                                'netIncome',
                              'netIncomeTTM',
                              'operatingCashFlow',
                              'operatingCashFlowTTM',
                              'dividendsPaid',
                              'dividendsPaidTTM',
                              'FCFF',
                              'FCFFTTM',
                              'goodwill'],
                        yline=['grossProfitRatio',
                               'grossProfitRatioTTM',
                                'operatingIncomeRatio',
                               'operatingIncomeRatioTTM',
                                'operatingIncomeAdjustedRatio',
                               'operatingIncomeAdjustedRatioTTM',
                                'netIncomeRatio',
                               'netIncomeRatioTTM'],
                                ydash=['Adj Close',
                                       'grossProfitRatioIndustryMedian',
                                    'operatingIncomeRatioIndustryMedian',
                                    'operatingIncomeAdjustedRatioIndustryMedian',
                                    'netIncomeRatioIndustryMedian',
                                    'CL=F_3M_Ave',
                                    'GC=F_3M_Ave',
                                    'GDP',
                                    'CPIAUCSL'],
                        width=1780, height=570,
                        title="Profitability")
    
    list_of_chart.append(fig_profit)
    ############################################ All Sample Yearly Metrics ###############################
    df_income_yearly = df_prices_fin.drop_duplicates(
        ['symbol','dateFinncialEnding'],keep='last').copy().groupby(
            ['symbol','industry','fiancialEndingYear'],dropna=False).agg(
        {'revenue': 'sum',
         'grossProfit':'sum',
         'operatingIncome':'sum',
         'operatingIncomeAfterTax':'sum',
         'operatingIncomeAdjusted':'sum',
         'operatingIncomeAdjustedAfterTax':'sum',
         'netIncome':'sum',
         'investedCapitalBookValueAdjusted':'mean'}).reset_index()                                                                                                              
    df_income_yearly["ROICFarbod"] = df_income_yearly['operatingIncomeAfterTax']/df_income_yearly['investedCapitalBookValueAdjusted']
    df_income_yearly["ROICAdjustedFarbod"] = df_income_yearly['operatingIncomeAdjustedAfterTax']/df_income_yearly['investedCapitalBookValueAdjusted']
    df_income_yearly['salesToCapitalRatioFarbod'] = df_income_yearly['revenue']/df_income_yearly['investedCapitalBookValueAdjusted']
    df_income_yearly['grossProfitRatio'] = df_income_yearly['grossProfit']/df_income_yearly['revenue']
    df_income_yearly['operatingIncomeRatio']=df_income_yearly['operatingIncome']/df_income_yearly['revenue']
    df_income_yearly['operatingIncomeAdjustedRatio']=df_income_yearly['operatingIncomeAdjusted']/df_income_yearly['revenue']
    df_income_yearly['netIncomeRatio']=df_income_yearly['netIncome']/df_income_yearly['revenue']
    df_income_yearly['symbol year'] = df_income_yearly['symbol'].astype(str) + " " +df_income_yearly['fiancialEndingYear'].astype(int).astype(str)
    
    ### Operating Income of All Sample
    fig_operating_income_all_sample = px.bar(df_income_yearly,
                x='symbol year',
                y=['operatingIncomeAdjustedRatio'],
        hover_data=['netIncome','operatingIncomeAdjusted','revenue'],
        color='symbol',
        text_auto='.2f')
    fig_operating_income_all_sample = fig_operating_income_all_sample.update_layout(width=1650,
                                                                                    height=520,
                                                                                    title='Operating Income Adjusted Ratio')
    list_of_chart.append(fig_operating_income_all_sample)
    ### Sales to Capital Ratio All Sample
    fig_sales_to_capital_all_sample = px.bar(df_income_yearly,
                                             x='symbol year',
                                             y=['salesToCapitalRatioFarbod'],
        hover_data=['investedCapitalBookValueAdjusted','revenue'],
        color='symbol',
        text_auto='.2f')
    fig_sales_to_capital_all_sample = fig_sales_to_capital_all_sample.update_layout(width=1650,
                                                                                    height=520,
                                                                                    title='Sales To Capital Farbod')
    list_of_chart.append(fig_sales_to_capital_all_sample)

    #####
    fig_roic_all_sample = px.bar(df_income_yearly,
                                             x='symbol year',
                                             y=['ROICAdjustedFarbod'],
        hover_data=['salesToCapitalRatioFarbod','revenue','investedCapitalBookValueAdjusted'],
        color='symbol',
        text_auto='.2f',
        barmode='stack')
    fig_roic_all_sample = fig_roic_all_sample.update_layout(width=1650,
                                                            height=520,
                                                            title='ROIC Adjusted')
    list_of_chart.append(fig_roic_all_sample)



    ###### Cost of Revenue
    fig_cost_of_revenue = plotly_line_dash_bar_chart(df_target_symbol_fin,
                x='dateFinncialEnding',
                ybar=['costOfRevenueRatio',
                   'sellingGeneralAndAdministrativeExpensesRatio',
                   'rndAmortizationRatio',
                   'InterestExpenseRatio',
                   'otherExpensesRatio'],
                   yline=['revenue'],
                   ydash=[],
                   title='COGS Ratio',
        barmode='stack',
        texttemplate= "%{value:.2f}"
        )
    fig_cost_of_revenue = fig_cost_of_revenue.update_layout(width=1650,
                                                            height=520,
                                                            title='Cost Of Revenue Ratio')
    list_of_chart.append(fig_cost_of_revenue)


    #### RIO
    fig_roi = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['totalLiabilitiesAndStockholdersEquityAdjustedTTMAve',
                                'investedCapitalBookValueAdjustedTTMAve',
                              'investedCapitalBookValueDebtEquityAdjustedTTMAve',
                                'totalStockholdersEquityTTMAve',
                                'cashAndShortTermInvestmentsTTMAve',
                                'totalDebt'],
                                yline=['salesToCapitalRatioFarbod','ROA','ROAAdjusted',
                                'ROIC','ROICAdjusted','ROICFarbod','ROICAdjustedFarbod','ROE','ROENetCash',
                                ],
                                ydash=['salesToCapitalRatioFarbodIndustryMedian','ROAIndustryMedian','ROAAdjustedIndustryMedian','ROICIndustryMedian',
                                    'ROICAdjustedIndustryMedian','ROICFarbodIndustryMedian','ROICAdjustedFarbodIndustryMedian','ROEIndustryMedian','ROENetCashIndustryMedian',
                                    ],
                                title='ROI',
                                width=1650, height=520)
    list_of_chart.append(fig_roi)

    ### Reinvestment
    fig_reinvestment = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['frimReinvestment','frimReinvestmentTTM',
                              'frimReinvestmentAdjusted','frimReinvestmentAdjustedTTM',],
                                yline=['frimReinvestmentRate','frimReinvestmentRateAdjusted',
                                       'frimReinvestmentRateTTM','frimReinvestmentRateAdjustedTTM'],
                                ydash=[],
                                title='Reinvestment',
                                width=1550, height=520)
    list_of_chart.append(fig_reinvestment)
    

    ### Growth Efficiency
    fig_growth_efficiency = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['revenue','revenueTTM',
                              'totalLiabilitiesAndStockholdersEquityAdjusted',
                              'totalLiabilitiesAndStockholdersEquityAdjustedTTMAve',
                              'investedCapitalBookValueAdjusted',
                              'investedCapitalBookValueAdjustedTTMAve',
                              'investedCapitalBookValueDebtEquityAdjusted',
                              'investedCapitalBookValueDebtEquityAdjustedTTMAve',
                              'investedCapitalBookValueDebtEquity',
                              'investedCapitalBookValueDebtEquityTTMAve',
                              'totalStockholdersEquity',
                              'totalDebt',
                              'cashAndShortTermInvestments',
                              'rndUnamortized',
                              'goodwill'],
                                yline=['salesToCapitalRatioFarbod','salesToCapitalRatio','salesToCapitalAdjustedRatio','averageSalesToCapitalRatio',
                                       'salesToCapitalRatioFarbodQ',
                                       'revenueYoY','revenueChange'],
                                ydash=['salesToCapitalRatioFarbodIndustryMedian',
                                       'salesToCapitalRatioFarbodQIndustryMedian',
                                       'salesToCapitalRatioFarbodQ'],
                                title='Revenue Growth Efficiency',
                                width=1550, height=550)
    list_of_chart.append(fig_growth_efficiency)


    ### cashflow
    fig_cashflow = plotly_line_dash_bar_chart(df_target_symbol_fin,
                        x='dateFinncialEnding',
                        ybar=['FCFF','FCFEPreDebt','FCFEAfterDebt',
                              'FCFFTTM','FCFEPreDebtTTM','FCFEAfterDebtTTM',
                              'netCashProvidedByOperatingActivities','operatingCashFlow','dividendsPaid','freeCashFlow',
                              'netCashProvidedByOperatingActivitiesTTM','operatingCashFlowTTM','dividendsPaidTTM','freeCashFlowTTM','frimReinvestment','frimReinvestmentTTM',
                              'netIncome','netIncomeTTM',
                              'totalDebt','netDebt'],
                                yline=['potentialPayOutRatio','potentialPayOutRatioTTM',
                                       'payOutRatio','payOutRatioTTM'],
                                ydash=[],
                                title='Cash Flow',
                                width=1550, height=500)
    list_of_chart.append(fig_cashflow)
    
    
    ### Scatter
    fig_rev_gross_profit = px.scatter(df_target_symbol_fin,
            x='revenue',
            y='grossProfit',
            hover_data=['grossProfitRatio','operatingIncomeRatio','operatingIncomeAdjustedRatio',
                        'netIncomeRatio','dateFinncialEndingHover'],
            color='grossProfitRatio',
            width=700,
            height=530)
    list_of_chart.append(fig_rev_gross_profit)


    fig_rev_operatingIncomeAdjusted = px.scatter(df_target_symbol_fin,
            x='revenue',
            y='operatingIncomeAdjusted',
            color='operatingIncomeAdjustedRatio',
            hover_data=['grossProfitRatio','operatingIncomeRatio',
                        'operatingIncomeAdjustedRatio','netIncomeRatio','dateFinncialEndingHover'],
            width=700,
            height=530)
    list_of_chart.append(fig_rev_operatingIncomeAdjusted)
    fig_rev_operatingIncome = px.scatter(df_target_symbol_fin,
            x='revenue',
            y='operatingIncome',
            color='operatingIncomeRatio',
            hover_data=['grossProfitRatio','operatingIncomeRatio',
                        'operatingIncomeAdjustedRatio','netIncomeRatio','dateFinncialEndingHover'],
            width=700,
            height=530)
    list_of_chart.append(fig_rev_operatingIncome)

    fig_rev_net_income = px.scatter(df_target_symbol_fin,
            x='revenue',
            y='netIncome',
            hover_data=['grossProfitRatio','operatingIncomeRatio','operatingIncomeAdjustedRatio',
                        'netIncomeRatio','dateFinncialEndingHover'],
            color='netIncomeRatio',
            width=700,
            height=530)
    
    list_of_chart.append(fig_rev_net_income)
    

    ######## Correlation Matrix
    import plotly.figure_factory as ff
    df_corr = df_target_symbol_fin[[
                                    'revenue','revenueTTM','operatingIncome','operatingIncomeTTM',
                                    'grossProfit','grossProfitTTM',
                                    'grossProfitRatio','grossProfitRatioTTM',
                                    'operatingIncomeRatio','operatingIncomeAdjustedRatio',
                                    'operatingIncomeRatioTTM','operatingIncomeAdjustedRatioTTM',
                                    'salesToCapitalRatioFarbod','salesToCapitalAdjustedRatio','salesToCapitalRatio',
                                    'revenueChange','revenueYoY','costOfRevenueYoY',
                                    'operatingIncomeYoY','operatingIncomeAdjustedYoY',
                                    'ROICAdjusted','ROICAdjustedFarbod',
                                    'costOfRevenue','costOfRevenueTTM',
                                    'researchAndDevelopmentExpenses',
                                    'sellingGeneralAndAdministrativeExpenses', 'otherExpenses',
                                    'operatingExpenses', 'costAndExpenses', 'interestExpense', 'depreciationAndAmortization',
                                    'frimReinvestmentRate',
        'CL=F_3M_Ave','GC=F_3M_Ave','GDP','CPIAUCSL']].corr(method='spearman')
    x = list(df_corr.columns)
    y = list(df_corr.index)
    z = np.array(df_corr)

    fig_corr_mat = ff.create_annotated_heatmap(
        z,
        x = x,
        y = y ,
        annotation_text = np.around(z, decimals=2),
        hoverinfo='x,y,z',
        colorscale=['#e83354','#f5edef','#68cc82'],
        font_colors= ['#000000']
        )
    fig_corr_mat = fig_corr_mat.update_layout(width=1530,height=670)
    list_of_chart.append(fig_corr_mat)



    list_of_col_for_valuation = ['Date','dateFinncialEnding','industry','symbol','Beta','RSquared','unleveredBetaSampleMedian','unleveredBetaIndustryMedian',
                                 'Adj Close','sharesOutstandingCurrentYahoo',
                                 'marketCap','marketCapConventional',
                                 'BetaIndustryMedian','RSquaredIndustryMedian','unleveredBetaIndustryMedian',
                             '10YearRF','DAAA','DBAA','costOfDebtBackwardLooking','GDP',
                             'revenue','operatingIncome','operatingIncomeAdjusted','operatingIncomeRatio','operatingIncomeAdjustedRatio',
                             'revenueTTM','operatingIncomeTTM','operatingIncomeAdjustedTTM',
                             'operatingIncomeRatioTTM','operatingIncomeAdjustedRatioTTM',
                             'effectiveTaxRate','effectiveTaxRateTTM','marginalTaxRate',
                             'salesToCapitalRatioFarbod','salesToCapitalAdjustedRatio','salesToCapitalRatio',
                             'salesToAssetRatio','salesToAssetAdjustedtRatio',
                             'ROICAdjusted','ROICAdjustedFarbod','frimReinvestmentRate',
                             'marketCapUSD','totalDebt','totalCash','cashAndCashEquivalents','shortTermInvestments','longTermInvestments']
    for chart in list_of_chart:
        try:
            chart.show()
        except:
            pass
    return(df_prices_fin[(df_prices_fin['symbol']==target_symbol)].drop_duplicates("dateFinncialEnding",keep='last')[list_of_col_for_valuation])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Valuation Describer

In [91]:
def valuation_describer(df_intc_valuation,
                        sharesOutstanding=1):
    """change the histrocgrame to  plogtly https://plotly.com/python/histograms/"""
    ### Get the Equity value at eahc percentile
    current_market_cap = df_intc_valuation['equity_value'].median()
    percentiles=np.arange(0, 110, 10)
    equity_value_at_each_percentile = np.percentile(df_intc_valuation['equity_valuation'], 
                                                    percentiles)
    equity_value_at_20_percentile=equity_value_at_each_percentile[2]
    equity_value_at_80_percentile=equity_value_at_each_percentile[8]
    df_valuation_res = pd.DataFrame({"percentiles":percentiles,
                                     "equity_value":equity_value_at_each_percentile})
    df_valuation_res['current_market_cap'] = current_market_cap
    df_valuation_res['current_price_per_share'] = current_market_cap/sharesOutstanding
    df_valuation_res['equity_value_per_share'] = df_valuation_res['equity_value']/sharesOutstanding
    df_valuation_res['Price/Value']= df_valuation_res['current_market_cap']/df_valuation_res['equity_value']
    df_valuation_res['PNL']= (df_valuation_res['equity_value']/df_valuation_res['current_market_cap'])-1
    ### Histogram
    fig = histogram_plotter_plotly(data=df_intc_valuation,
                              colmn_name ='equity_valuation',
                              xlabel ='Market Cap',
                              title='Intrinsic Equity Value Distribution',
                              bins=200,
                              percentile=[15,50,85],
                              color=['green','yellow','red'],
                              histnorm='percent',
                              height=510,
                              width=720)
    fig = fig.add_vline(x = current_market_cap, line_dash = 'dash',line_color='black',
                        annotation_text="-Current Market Cap",
                        annotation_font_size=10)
    # 
    ### Plot cummultaive distribution of intrincsict equity value
    fig_cdf = ecdf_plotter_plotly(data=df_intc_valuation,
                              colmn_name ='equity_valuation',
                              xlabel ='Market Cap',
                              title='Intrinsic Equity Value Cumulative Distribution',
                              percentile=[15,50,85],
                              color=['green','yellow','red'],
                              marginal='histogram',
                              height=510,
                              width=720)
    fig_cdf = fig_cdf.add_vline(x = current_market_cap, line_dash = 'dash',line_color='black',annotation_text="-Current Market Cap", annotation_font_size=10)
    #fig_cdf.show()
    ### Model Correlation Chart
    fig_model_correlation_chart = px.bar(df_intc_valuation.rename(columns=dict(zip(df_intc_valuation.columns,
                                     [c.replace("_"," ") for c in df_intc_valuation.columns]))).corr(method='pearson')[['equity valuation']].sort_values(
                                         "equity valuation",ascending=False).reset_index(),
       x='index',
       y='equity valuation',
       title='Model Variable Pearson Correlation with Equity Intrinsic Value',
        height=730,
       width=1600,
       text_auto='.2f',
       labels={'index':'Model Variable',
               'equity valuation':'Correlation'})
    fig_model_correlation_chart.show()
    for col in ['revenue_growth_rate_cycle1_begin',
                'revenue_growth_rate_cycle1_end',
                'revenue_growth_rate_cycle2_begin',
                'revenue_growth_rate_cycle2_end',
                'revenue_growth_rate_cycle3_begin',
                'revenue_growth_rate_cycle3_end'
                'revenue_growth_rate',
                'risk_free_rate','ERP',
                'additional_return_on_cost_of_capital_in_perpetuity',
                'terminal_reinvestmentRate',
                'terminal_afterTaxOperatingIncome',
                'firm_valuation',
                'terminal_FCFF',
                'unlevered_beta',
                'terminal_unlevered_beta',
                'current_sales_to_capital_ratio',
                'terminal_sales_to_capital_ratio',
                'current_operating_margin',
                'terminal_operating_margin',
                'terminal_revenue']:
                try:
                    histogram_plotter_plotly(df_intc_valuation,
                                             colmn_name=col,
                                             xlabel=col.replace("_"," "),
                                             bins=200).show()
                except:
                    pass
    fig_cdf.show()
    fig.show()
    return(df_valuation_res)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Example

In [92]:
list_of_comparables= ['GOOG','META','AMZN','MSFT']

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [93]:
%%time
output_price_and_financials = historical_price_and_financials(
                                                              list_of_comparables,
                                         market_index='SPY',
                                         resample=True,
                                         resample_interval='5B',
                                         window=300,
                                         market_cap=True,
                                         rnd_amortization_period_begin=1,
                                         rnd_amortization_period=3,
                                         fmp_api = my_fmp_api,
                                         fred_api = fred_key_api)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

loading companies profile from FMP
4  Companies profile retrieved from FMP
Getting currency exchange ticker symbol adj close price
Retriving Exchange Rates from FRED
Retriving Exchange Rates from FRED Complete (13630, 22)
Retriving Exchange Rates from YahooFinance
[*********************100%***********************]  30 of 30 completed

30 Failed downloads:
- CAD=X: No data found for this date range, symbol may be delisted
- MYR=X: No data found for this date range, symbol may be delisted
- GBP=X: No data found for this date range, symbol may be delisted
- RUB=X: No data found for this date range, symbol may be delisted
- NZD=X: No data found for this date range, symbol may be delisted
- JPY=X: No data found for this date range, symbol may be delisted
- DKK=X: No data found for this date range, symbol may be delisted
- NOK=X: No data found for this date range, symbol may be delisted
- BRL=X: No data found for this date range, symbol may be delisted
- INR=X: No data found for this date ra

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
df_prices_fin = output_price_and_financials['prices_and_financials'].copy()
df_prices_fin.tail(3)

<IPython.core.display.Javascript object>

NameError: name 'output_price_and_financials' is not defined

### Analyze Fundamentals

In [ ]:
company_analyzer(df_prices_fin,
                     target_symbol = 'META',
                     date=dt.datetime(2010,1,1)).tail()

<IPython.core.display.Javascript object>

NameError: name 'df_prices_fin' is not defined

## **Narrative and Numbers**

1. META will be transformed from its current state of being a platform focus on social media, to being a hardware and software provider in augmented reality industry for commercial purposes. 
2. 'Facebook' will be a declining product, a medium funding this new VR project.
3. Revenue will be declining for the next 4 years, and "Facebook" will make a comeback as Meta. 
4. The margin will be shrinked as Meta gets larger revenue from selling VR at cost or lost, and make money by monitizing the platfrom via subscription or IAP service fee or providing infostructure to other business to use metaverse.

##### Point Estimate of the firm value

In [ ]:
base_case_valuatio = valuator_multi_phase(
            risk_free_rate =.038,
            ERP = .057,
            current_pretax_cost_of_debt = .057,
            terminal_pretax_cost_of_debt =.04,
            year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt = 4,
            equity_value = 377,
            debt_value = 16.067,
            cash_and_non_operating_asset= 46.044,
            unlevered_beta = 1.45, #### high beta due to VR and metaverse is a big startup vc funded by Facebook
            terminal_unlevered_beta =1.18,
            year_beta_begins_to_converge_to_terminal_beta = 3, ### early conversion since Facebook backs it up and monitzie it sooner than regular start ups
            current_effective_tax_rate =.15,
            marginal_tax_rate =.24,
            year_effective_tax_rate_begin_to_converge_marginal_tax_rate = 5,
             revenue_base = 117,
             revenue_growth_rate_cycle1_begin = -.12,
             revenue_growth_rate_cycle1_end = -.03,
             length_of_cylcle1=4,
             revenue_growth_rate_cycle2_begin = .03,
             revenue_growth_rate_cycle2_end = .18,
             length_of_cylcle2=5,
             revenue_growth_rate_cycle3_begin = .16,
             revenue_growth_rate_cycle3_end = .03,
             length_of_cylcle3=5,
            revenue_convergance_periods_cycle1= 1,
            revenue_convergance_periods_cycle2=1,
            revenue_convergance_periods_cycle3=1,
            current_sales_to_capital_ratio = .85,
            terminal_sales_to_capital_ratio = 1,
            year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital = 3,
            current_operating_margin = .44,
            terminal_operating_margin =.36,
            year_operating_margin_begins_to_converge_to_terminal_operating_margin = 3,
            additional_return_on_cost_of_capital_in_perpetuity=0.04,
             asset_liquidation_during_negative_growth=0.00)
print('value of operating assets',np.round(base_case_valuatio['value_of_operating_assets'],2),'\n',
      'cash and non operating asset',np.round(base_case_valuatio['cash_and_non_operating_asset'],2),'\n',
      'debt value',np.round(base_case_valuatio['debt_value'],2),'\n',
      'firm value',np.round(base_case_valuatio['firm_value'],2),'\n',
      'Intrinsic Equity value',"${:.2f}".format(np.round(base_case_valuatio['equity_value'],2)))
base_case_valuatio['valuation'].reset_index()

<IPython.core.display.Javascript object>

value of operating assets 364.9 
 cash and non operating asset 46.04 
 debt value 16.07 
 firm value 410.95 
 Intrinsic Equity value $394.88


,index,cumWACC,cumCostOfEquity,beta,revenueGrowth,revneues,margins,ebit,sales_to_capital_ratio,taxRate,afterTaxOperatingIncome,reinvestment,FCFF,ERP,reinvestmentRate,PVFCFF
0,0,1.12,1.12,1.50,-0.12,102.96,0.44,45.30,0.85,0.15,38.51,-0.00,38.51,0.06,-0.00,34.38
1,1,1.25,1.26,1.50,-0.09,93.69,0.44,41.23,0.85,0.15,35.04,-0.00,35.04,0.06,-0.00,27.93
2,2,1.41,1.42,1.50,-0.06,88.07,0.44,38.75,0.85,0.15,32.94,-0.00,32.94,0.06,-0.00,23.44
3,3,1.57,1.59,1.47,-0.03,85.43,0.43,36.97,0.86,0.15,31.42,-0.00,31.42,0.06,-0.00,19.99
4,4,1.76,1.78,1.45,0.03,87.99,0.43,37.44,0.88,0.15,31.82,2.92,28.90,0.06,0.09,16.46
5,5,1.96,1.99,1.42,0.07,93.93,0.42,39.28,0.89,0.16,33.00,6.67,26.33,0.06,0.20,13.44
6,6,2.18,2.23,1.40,0.10,103.80,0.41,42.65,0.90,0.17,35.40,10.90,24.50,0.06,0.31,11.22
7,7,2.43,2.49,1.37,0.14,118.59,0.40,47.87,0.92,0.18,39.25,16.11,23.14,0.06,0.41,9.52
8,8,2.70,2.77,1.34,0.18,139.93,0.40,55.46,0.93,0.19,44.93,22.91,22.02,0.06,0.51,8.15
9,9,3.00,3.09,1.32,0.16,162.32,0.39,63.16,0.95,0.20,50.53,23.68,26.85,0.06,0.47,8.95


### Monte Carlo Valuation

In [ ]:
df_valuation = df_prices_fin[(df_prices_fin['symbol']=='META') & 
              (df_prices_fin['Date'] == dt.datetime(2022,9,23))].groupby(['Date','symbol']).apply(lambda row:
                                                                                                monte_carlo_valuator_multi_phase(
    risk_free_rate = ot.Normal(row['10YearRF'].values[0],.003),
    ERP =  ot.Normal(0.057,.001) ,
    equity_value = ot.Triangular(row['marketCapUSD'].values[0]*.95,row['marketCapUSD'].values[0],row['marketCapUSD'].values[0]*1.05),
    debt_value = ot.Triangular(row['totalDebt'].values[0]*.99,row['totalDebt'].values[0],row['totalDebt'].values[0]*1.01),
    unlevered_beta = ot.Triangular(1.3,1.45,1.6),
    terminal_unlevered_beta = ot.Triangular(1.1,1.18,1.26),
    year_beta_begins_to_converge_to_terminal_beta = ot.Uniform(2,4),
    current_pretax_cost_of_debt = ot.Triangular(row['DAAA'].values[0]*.8,
                                                row['DAAA'].values[0],
                                                row['DAAA'].values[0]*1.2),
    terminal_pretax_cost_of_debt = ot.Triangular(.04,.045,.05),
    year_cost_of_debt_begins_to_converge_to_terminal_cost_of_debt = ot.Uniform(2,4),
    current_effective_tax_rate = ot.Triangular(row['effectiveTaxRateTTM'].values[0]*.8,
                                               row['effectiveTaxRateTTM'].values[0],
                                               row['effectiveTaxRateTTM'].values[0]*1.1) ,
    marginal_tax_rate = ot.Triangular(row['marginalTaxRate'].values[0]*.9,row['marginalTaxRate'].values[0]*.93,row['marginalTaxRate'].values[0]*.97),
    year_effective_tax_rate_begin_to_converge_marginal_tax_rate = ot.Uniform(4,6),
     revenue_base = ot.Triangular(row['revenueTTM'].values[0]*.98,row['revenueTTM'].values[0]*.99,row['revenueTTM'].values[0]*.999),
     revenue_growth_rate_cycle1_begin = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm(-0.5, loc=-.105, scale=.023))), ### Negatively skewed bc maybe the worst is yet to come
     revenue_growth_rate_cycle1_end = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm( -0.6 ,loc=-.03, scale=.008))),
     revenue_growth_rate_cycle2_begin = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm(0.8, loc=.03, scale=.007))),
     revenue_growth_rate_cycle2_end = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm(.6, loc=.175, scale=.028))), ### Posetively skewed bc maybe  I might be understimating how big Metaverse can generate money Mark Zuckerburg
     revenue_growth_rate_cycle3_begin = ot.Distribution(ot.SciPyDistribution(scipy.stats.skewnorm(.7, loc=.165, scale=.024))),
     revenue_growth_rate_cycle3_end = ot.Triangular(.024,.028,0.032),
    revenue_convergance_periods_cycle1 = ot.Uniform(1,2),
    revenue_convergance_periods_cycle2 = ot.Uniform(1,2),
    revenue_convergance_periods_cycle3 = ot.Uniform(1,2),
    length_of_cylcle1 = ot.Uniform(4,6),
    length_of_cylcle2 = ot.Uniform(4,7),
    length_of_cylcle3 = ot.Uniform(4,7),
    current_sales_to_capital_ratio = ot.Triangular(.6,.85,1.2),
    terminal_sales_to_capital_ratio = ot.Triangular(.6,1,1.3),
    year_sales_to_capital_begins_to_converge_to_terminal_sales_to_capital = ot.Uniform(3,5),
    current_operating_margin = ot.Uniform(.41,.46),
    terminal_operating_margin = ot.Triangular(.22,.36,.52),
    year_operating_margin_begins_to_converge_to_terminal_operating_margin = ot.Uniform(2,5),
    additional_return_on_cost_of_capital_in_perpetuity = ot.Triangular(0.03,0.04,0.05), ### Facebook will continue to reinvest well and earn higher return than thier cost of capital
    cash_and_non_operating_asset = ot.Uniform(row['totalCash'].values[0],row['totalCash'].values[0]+1),
    asset_liquidation_during_negative_growth = ot.Uniform(0,0.0000001),
    sample_size = 5000,
    list_of_correlation_between_variables=[['additional_return_on_cost_of_capital_in_perpetuity','terminal_sales_to_capital_ratio',0.4],
                                           ['additional_return_on_cost_of_capital_in_perpetuity','terminal_operating_margin',.6],
                                           ['revenue_growth_rate_cycle3_begin','length_of_cylcle2',-.2],
                                           ['terminal_operating_margin','terminal_sales_to_capital_ratio',-0.2]
                                           ])
                                                                                                )

<IPython.core.display.Javascript object>

Scenario Generation Complete


In [ ]:
valuation_describer(df_valuation,sharesOutstanding=2225760000)

Output hidden; open in https://colab.research.google.com to view.